In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW14OT-MakeConcordance-2Chronicles.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""
import string

from BibleWordList2 import *

SecondChronicles01 = [
    ['2Chronicles','2Chr','01','001','1',"And Solomon the son of David was strengthened in his kingdom, and the LORD his God \emph{was} with him, and magnified him exceedingly."],
['2Chronicles','2Chr','01','002','2',"Then Solomon spake unto all Israel, to the captains of thousands and of hundreds, and to the judges, and to every governor in all Israel, the chief of the fathers."],
['2Chronicles','2Chr','01','003','3',"So Solomon, and all the congregation with him, went to the high place that \emph{was} at Gibeon; for there was the tabernacle of the congregation of God, which Moses the servant of the LORD had made in the wilderness."],
['2Chronicles','2Chr','01','004','4',"But the ark of God had David brought up from Kirjath-jearim to \emph{the} \emph{place} \emph{which} David had prepared for it: for he had pitched a tent for it at Jerusalem."],
['2Chronicles','2Chr','01','005','5',"Moreover the brasen altar, that Bezaleel the son of Uri, the son of Hur, had made, he put before the tabernacle of the LORD: and Solomon and the congregation sought unto it."],
['2Chronicles','2Chr','01','006','6',"And Solomon went up thither to the brasen altar before the LORD, which \emph{was} at the tabernacle of the congregation, and offered a thousand burnt offerings upon it."],
['2Chronicles','2Chr','01','007','7',"In that night did God appear unto Solomon, and said unto him, Ask what I shall give thee."],
['2Chronicles','2Chr','01','008','8',"And Solomon said unto God, Thou hast shewed great mercy unto David my father, and hast made me to reign in his stead."],
['2Chronicles','2Chr','01','009','9',"Now, O LORD God, let thy promise unto David my father be established: for thou hast made me king over a people like the dust of the earth in multitude."],
['2Chronicles','2Chr','01','010','10',"Give me now wisdom and knowledge, that I may go out and come in before this people: for who can judge this thy people, \emph{that} \emph{is} \emph{so} great?"],
['2Chronicles','2Chr','01','011','11',"And God said to Solomon, Because this was in thine heart, and thou hast not asked riches, wealth, or honour, nor the life of thine enemies, neither yet hast asked long life; but hast asked wisdom and knowledge for thyself, that thou mayest judge my people, over whom I have made thee king:"],
['2Chronicles','2Chr','01','012','12',"Wisdom and knowledge \emph{is} granted unto thee; and I will give thee riches, and wealth, and honour, such as none of the kings have had that \emph{have} \emph{been} before thee, neither shall there any after thee have the like."],
['2Chronicles','2Chr','01','013','13',"Then Solomon came \emph{from} \emph{his} \emph{journey} to the high place that \emph{was} at Gibeon to Jerusalem, from before the tabernacle of the congregation, and reigned over Israel."],
['2Chronicles','2Chr','01','014','14',"And Solomon gathered chariots and horsemen: and he had a thousand and four hundred chariots, and twelve thousand horsemen, which he placed in the chariot cities, and with the king at Jerusalem."],
['2Chronicles','2Chr','01','015','15',"And the king made silver and gold at Jerusalem \emph{as} \emph{plenteous} as stones, and cedar trees made he as the sycomore trees that \emph{are} in the vale for abundance."],
['2Chronicles','2Chr','01','016','16',"And Solomon had horses brought out of Egypt, and linen yarn: the king's merchants received the linen yarn at a price."],
['2Chronicles','2Chr','01','017','17',"And they fetched up, and brought forth out of Egypt a chariot for six hundred \emph{shekels} of silver, and an horse for an hundred and fifty: and so brought they out \emph{horses} for all the kings of the Hittites, and for the kings of Syria, by their means."]
]

SecondChronicles02 = [
    ['2Chronicles','2Chr','02','001','1',"And Solomon determined to build an house for the name of the LORD, and an house for his kingdom."],
['2Chronicles','2Chr','02','002','2',"And Solomon told out threescore and ten thousand men to bear burdens, and fourscore thousand to hew in the mountain, and three thousand and six hundred to oversee them."],
['2Chronicles','2Chr','02','003','3',"And Solomon sent to Huram the king of Tyre, saying, As thou didst deal with David my father, and didst send him cedars to build him an house to dwell therein, \emph{even} \emph{so} \emph{deal} \emph{with} \emph{me}."],
['2Chronicles','2Chr','02','004','4',"Behold, I build an house to the name of the LORD my God, to dedicate \emph{it} to him, \emph{and} to burn before him sweet incense, and for the continual shewbread, and for the burnt offerings morning and evening, on the sabbaths, and on the new moons, and on the solemn feasts of the LORD our God. This \emph{is} \emph{an} \emph{ordinance} for ever to Israel."],
['2Chronicles','2Chr','02','005','5',"And the house which I build \emph{is} great: for great \emph{is} our God above all gods."],
['2Chronicles','2Chr','02','006','6',"But who is able to build him an house, seeing the heaven and heaven of heavens cannot contain him? who \emph{am} I then, that I should build him an house, save only to burn sacrifice before him?"],
['2Chronicles','2Chr','02','007','7',"Send me now therefore a man cunning to work in gold, and in silver, and in brass, and in iron, and in purple, and crimson, and blue, and that can skill to grave with the cunning men that \emph{are} with me in Judah and in Jerusalem, whom David my father did provide."],
['2Chronicles','2Chr','02','008','8',"Send me also cedar trees, fir trees, and algum trees, out of Lebanon: for I know that thy servants can skill to cut timber in Lebanon; and, behold, my servants \emph{shall} \emph{be} with thy servants,"],
['2Chronicles','2Chr','02','009','9',"Even to prepare me timber in abundance: for the house which I am about to build \emph{shall} \emph{be} wonderful great."],
['2Chronicles','2Chr','02','010','10',"And, behold, I will give to thy servants, the hewers that cut timber, twenty thousand measures of beaten wheat, and twenty thousand measures of barley, and twenty thousand baths of wine, and twenty thousand baths of oil."],
['2Chronicles','2Chr','02','011','11',"Then Huram the king of Tyre answered in writing, which he sent to Solomon, Because the LORD hath loved his people, he hath made thee king over them."],
['2Chronicles','2Chr','02','012','12',"Huram said moreover, Blessed \emph{be} the LORD God of Israel, that made heaven and earth, who hath given to David the king a wise son, endued with prudence and understanding, that might build an house for the LORD, and an house for his kingdom."],
['2Chronicles','2Chr','02','013','13',"And now I have sent a cunning man, endued with understanding, of Huram my father's,"],
['2Chronicles','2Chr','02','014','14',"The son of a woman of the daughters of Dan, and his father \emph{was} a man of Tyre, skilful to work in gold, and in silver, in brass, in iron, in stone, and in timber, in purple, in blue, and in fine linen, and in crimson; also to grave any manner of graving, and to find out every device which shall be put to him, with thy cunning men, and with the cunning men of my lord David thy father."],
['2Chronicles','2Chr','02','015','15',"Now therefore the wheat, and the barley, the oil, and the wine, which my lord hath spoken of, let him send unto his servants:"],
['2Chronicles','2Chr','02','016','16',"And we will cut wood out of Lebanon, as much as thou shalt need: and we will bring it to thee in flotes by sea to Joppa; and thou shalt carry it up to Jerusalem."],
['2Chronicles','2Chr','02','017','17',"And Solomon numbered all the strangers that \emph{were} in the land of Israel, after the numbering wherewith David his father had numbered them; and they were found an hundred and fifty thousand and three thousand and six hundred."],
['2Chronicles','2Chr','02','018','18',"And he set threescore and ten thousand of them \emph{to} \emph{be} bearers of burdens, and fourscore thousand \emph{to} \emph{be} hewers in the mountain, and three thousand and six hundred overseers to set the people a work."]
]

SecondChronicles03 = [
    ['2Chronicles','2Chr','03','001','1',"Then Solomon began to build the house of the LORD at Jerusalem in mount Moriah, where \emph{the} \emph{LORD} appeared unto David his father, in the place that David had prepared in the threshingfloor of Ornan the Jebusite."],
['2Chronicles','2Chr','03','002','2',"And he began to build in the second \emph{day} of the second month, in the fourth year of his reign."],
['2Chronicles','2Chr','03','003','3',"Now these \emph{are} \emph{the} \emph{things} \emph{wherein} Solomon was instructed for the building of the house of God. The length by cubits after the first measure \emph{was} threescore cubits, and the breadth twenty cubits."],
['2Chronicles','2Chr','03','004','4',"And the porch that \emph{was} in the front \emph{of} \emph{the} \emph{house}, the length \emph{of} \emph{it} \emph{was} according to the breadth of the house, twenty cubits, and the height \emph{was} an hundred and twenty: and he overlaid it within with pure gold."],
['2Chronicles','2Chr','03','005','5',"And the greater house he cieled with fir tree, which he overlaid with fine gold, and set thereon palm trees and chains."],
['2Chronicles','2Chr','03','006','6',"And he garnished the house with precious stones for beauty: and the gold \emph{was} gold of Parvaim."],
['2Chronicles','2Chr','03','007','7',"He overlaid also the house, the beams, the posts, and the walls thereof, and the doors thereof, with gold; and graved cherubims on the walls."],
['2Chronicles','2Chr','03','008','8',"And he made the most holy house, the length whereof \emph{was} according to the breadth of the house, twenty cubits, and the breadth thereof twenty cubits: and he overlaid it with fine gold, \emph{amounting} to six hundred talents."],
['2Chronicles','2Chr','03','009','9',"And the weight of the nails \emph{was} fifty shekels of gold. And he overlaid the upper chambers with gold."],
['2Chronicles','2Chr','03','010','10',"And in the most holy house he made two cherubims of image work, and overlaid them with gold."],
['2Chronicles','2Chr','03','011','11',"And the wings of the cherubims \emph{were} twenty cubits long: one wing \emph{of} \emph{the} \emph{one} \emph{cherub} \emph{was} five cubits, reaching to the wall of the house: and the other wing \emph{was} \emph{likewise} five cubits, reaching to the wing of the other cherub."],
['2Chronicles','2Chr','03','012','12',"And \emph{one} wing of the other cherub \emph{was} five cubits, reaching to the wall of the house: and the other wing \emph{was} five cubits \emph{also}, joining to the wing of the other cherub."],
['2Chronicles','2Chr','03','013','13',"The wings of these cherubims spread themselves forth twenty cubits: and they stood on their feet, and their faces \emph{were} inward."],
['2Chronicles','2Chr','03','014','14',"And he made the vail \emph{of} blue, and purple, and crimson, and fine linen, and wrought cherubims thereon."],
['2Chronicles','2Chr','03','015','15',"Also he made before the house two pillars of thirty and five cubits high, and the chapiter that \emph{was} on the top of each of them \emph{was} five cubits."],
['2Chronicles','2Chr','03','016','16',"And he made chains, \emph{as} in the oracle, and put \emph{them} on the heads of the pillars; and made an hundred pomegranates, and put \emph{them} on the chains."],
['2Chronicles','2Chr','03','017','17',"And he reared up the pillars before the temple, one on the right hand, and the other on the left; and called the name of that on the right hand Jachin, and the name of that on the left Boaz."]
]

SecondChronicles04 = [
    ['Chronicles','2Chr','04','001','1',"Moreover he made an altar of brass, twenty cubits the length thereof, and twenty cubits the breadth thereof, and ten cubits the height thereof."],
['Chronicles','2Chr','04','002','2',"Also he made a molten sea of ten cubits from brim to brim, round in compass, and five cubits the height thereof; and a line of thirty cubits did compass it round about."],
['Chronicles','2Chr','04','003','3',"And under it \emph{was} the similitude of oxen, which did compass it round about: ten in a cubit, compassing the sea round about. Two rows of oxen \emph{were} cast, when it was cast."],
['Chronicles','2Chr','04','004','4',"It stood upon twelve oxen, three looking toward the north, and three looking toward the west, and three looking toward the south, and three looking toward the east: and the sea \emph{was} \emph{set} above upon them, and all their hinder parts \emph{were} inward."],
['Chronicles','2Chr','04','005','5',"And the thickness of it \emph{was} an handbreadth, and the brim of it like the work of the brim of a cup, with flowers of lilies; \emph{and} it received and held three thousand baths."],
['Chronicles','2Chr','04','006','6',"He made also ten lavers, and put five on the right hand, and five on the left, to wash in them: such things as they offered for the burnt offering they washed in them; but the sea \emph{was} for the priests to wash in."],
['Chronicles','2Chr','04','007','7',"And he made ten candlesticks of gold according to their form, and set \emph{them} in the temple, five on the right hand, and five on the left."],
['Chronicles','2Chr','04','008','8',"He made also ten tables, and placed \emph{them} in the temple, five on the right side, and five on the left. And he made an hundred basons of gold."],
['Chronicles','2Chr','04','009','9',"Furthermore he made the court of the priests, and the great court, and doors for the court, and overlaid the doors of them with brass."],
['Chronicles','2Chr','04','010','10',"And he set the sea on the right side of the east end, over against the south."],
['Chronicles','2Chr','04','011','11',"And Huram made the pots, and the shovels, and the basons. And Huram finished the work that he was to make for king Solomon for the house of God;"],
['Chronicles','2Chr','04','012','12',"\emph{To} \emph{wit}, the two pillars, and the pommels, and the chapiters \emph{which} \emph{were} on the top of the two pillars, and the two wreaths to cover the two pommels of the chapiters which \emph{were} on the top of the pillars;"],
['Chronicles','2Chr','04','013','13',"And four hundred pomegranates on the two wreaths; two rows of pomegranates on each wreath, to cover the two pommels of the chapiters which \emph{were} upon the pillars."],
['Chronicles','2Chr','04','014','14',"He made also bases, and lavers made he upon the bases;"],
['Chronicles','2Chr','04','015','15',"One sea, and twelve oxen under it."],
['Chronicles','2Chr','04','016','16',"The pots also, and the shovels, and the fleshhooks, and all their instruments, did Huram his father make to king Solomon for the house of the LORD of bright brass."],
['Chronicles','2Chr','04','017','17',"In the plain of Jordan did the king cast them, in the clay ground between Succoth and Zeredathah."],
['Chronicles','2Chr','04','018','18',"Thus Solomon made all these vessels in great abundance: for the weight of the brass could not be found out."],
['Chronicles','2Chr','04','019','19',"And Solomon made all the vessels that \emph{were} \emph{for} the house of God, the golden altar also, and the tables whereon the shewbread \emph{was} \emph{set};"],
['Chronicles','2Chr','04','020','20',"Moreover the candlesticks with their lamps, that they should burn after the manner before the oracle, of pure gold;"],
['Chronicles','2Chr','04','021','21',"And the flowers, and the lamps, and the tongs, \emph{made} \emph{he} \emph{of} gold, \emph{and} that perfect gold;"],
['Chronicles','2Chr','04','022','22',"And the snuffers, and the basons, and the spoons, and the censers, \emph{of} pure gold: and the entry of the house, the inner doors thereof for the most holy \emph{place}, and the doors of the house of the temple, \emph{were} \emph{of} gold."]
]

SecondChronicles05 = [
    ['Chronicles','2Chr','05','001','1',"Thus all the work that Solomon made for the house of the LORD was finished: and Solomon brought in \emph{all} the things that David his father had dedicated; and the silver, and the gold, and all the instruments, put he among the treasures of the house of God."],
['Chronicles','2Chr','05','002','2',"Then Solomon assembled the elders of Israel, and all the heads of the tribes, the chief of the fathers of the children of Israel, unto Jerusalem, to bring up the ark of the covenant of the LORD out of the city of David, which \emph{is} Zion."],
['Chronicles','2Chr','05','003','3',"Wherefore all the men of Israel assembled themselves unto the king in the feast which \emph{was} in the seventh month."],
['Chronicles','2Chr','05','004','4',"And all the elders of Israel came; and the Levites took up the ark."],
['Chronicles','2Chr','05','005','5',"And they brought up the ark, and the tabernacle of the congregation, and all the holy vessels that \emph{were} in the tabernacle, these did the priests \emph{and} the Levites bring up."],
['Chronicles','2Chr','05','006','6',"Also king Solomon, and all the congregation of Israel that were assembled unto him before the ark, sacrificed sheep and oxen, which could not be told nor numbered for multitude."],
['Chronicles','2Chr','05','007','7',"And the priests brought in the ark of the covenant of the LORD unto his place, to the oracle of the house, into the most holy \emph{place,} \emph{even} under the wings of the cherubims:"],
['Chronicles','2Chr','05','008','8',"For the cherubims spread forth \emph{their} wings over the place of the ark, and the cherubims covered the ark and the staves thereof above."],
['Chronicles','2Chr','05','009','9',"And they drew out the staves \emph{of} \emph{the} \emph{ark}, that the ends of the staves were seen from the ark before the oracle; but they were not seen without. And there it is unto this day."],
['Chronicles','2Chr','05','010','10',"\emph{There} \emph{was} nothing in the ark save the two tables which Moses put \emph{therein} at Horeb, when the LORD made \emph{a} \emph{covenant} with the children of Israel, when they came out of Egypt."],
['Chronicles','2Chr','05','011','11',"And it came to pass, when the priests were come out of the holy \emph{place}: (for all the priests \emph{that} \emph{were} present were sanctified, \emph{and} did not \emph{then} wait by course:"],
['Chronicles','2Chr','05','012','12',"Also the Levites \emph{which} \emph{were} the singers, all of them of Asaph, of Heman, of Jeduthun, with their sons and their brethren, \emph{being} arrayed in white linen, having cymbals and psalteries and harps, stood at the east end of the altar, and with them an hundred and twenty priests sounding with trumpets:)"],
['Chronicles','2Chr','05','013','13',"It came even to pass, as the trumpeters and singers \emph{were} as one, to make one sound to be heard in praising and thanking the LORD; and when they lifted up \emph{their} voice with the trumpets and cymbals and instruments of musick, and praised the LORD, \emph{saying}, For \emph{he} \emph{is} good; for his mercy \emph{endureth} for ever: that \emph{then} the house was filled with a cloud, \emph{even} the house of the LORD;"],
['Chronicles','2Chr','05','014','14',"So that the priests could not stand to minister by reason of the cloud: for the glory of the LORD had filled the house of God."]
]

SecondChronicles06 = [
    ['Chronicles','2Chr','06','001','1',"Then said Solomon, The LORD hath said that he would dwell in the thick darkness."],
['Chronicles','2Chr','06','002','2',"But I have built an house of habitation for thee, and a place for thy dwelling for ever."],
['Chronicles','2Chr','06','003','3',"And the king turned his face, and blessed the whole congregation of Israel: and all the congregation of Israel stood."],
['Chronicles','2Chr','06','004','4',"And he said, Blessed \emph{be} the LORD God of Israel, who hath with his hands fulfilled \emph{that} which he spake with his mouth to my father David, saying,"],
['Chronicles','2Chr','06','005','5',"Since the day that I brought forth my people out of the land of Egypt I chose no city among all the tribes of Israel to build an house in, that my name might be there; neither chose I any man to be a ruler over my people Israel:"],
['Chronicles','2Chr','06','006','6',"But I have chosen Jerusalem, that my name might be there; and have chosen David to be over my people Israel."],
['Chronicles','2Chr','06','007','7',"Now it was in the heart of David my father to build an house for the name of the LORD God of Israel."],
['Chronicles','2Chr','06','008','8',"But the LORD said to David my father, Forasmuch as it was in thine heart to build an house for my name, thou didst well in that it was in thine heart:"],
['Chronicles','2Chr','06','009','9',"Notwithstanding thou shalt not build the house; but thy son which shall come forth out of thy loins, he shall build the house for my name."],
['Chronicles','2Chr','06','010','10',"The LORD therefore hath performed his word that he hath spoken: for I am risen up in the room of David my father, and am set on the throne of Israel, as the LORD promised, and have built the house for the name of the LORD God of Israel."],
['Chronicles','2Chr','06','011','11',"And in it have I put the ark, wherein \emph{is} the covenant of the LORD, that he made with the children of Israel."],
['Chronicles','2Chr','06','012','12',"And he stood before the altar of the LORD in the presence of all the congregation of Israel, and spread forth his hands:"],
['Chronicles','2Chr','06','013','13',"For Solomon had made a brasen scaffold, of five cubits long, and five cubits broad, and three cubits high, and had set it in the midst of the court: and upon it he stood, and kneeled down upon his knees before all the congregation of Israel, and spread forth his hands toward heaven,"],
['Chronicles','2Chr','06','014','14',"And said, O LORD God of Israel, \emph{there} \emph{is} no God like thee in the heaven, nor in the earth; which keepest covenant, and \emph{shewest} mercy unto thy servants, that walk before thee with all their hearts:"],
['Chronicles','2Chr','06','015','15',"Thou which hast kept with thy servant David my father that which thou hast promised him; and spakest with thy mouth, and hast fulfilled \emph{it} with thine hand, as \emph{it} \emph{is} this day."],
['Chronicles','2Chr','06','016','16',"Now therefore, O LORD God of Israel, keep with thy servant David my father that which thou hast promised him, saying, There shall not fail thee a man in my sight to sit upon the throne of Israel; yet so that thy children take heed to their way to walk in my law, as thou hast walked before me."],
['Chronicles','2Chr','06','017','17',"Now then, O LORD God of Israel, let thy word be verified, which thou hast spoken unto thy servant David."],
['Chronicles','2Chr','06','018','18',"But will God in very deed dwell with men on the earth? behold, heaven and the heaven of heavens cannot contain thee; how much less this house which I have built!"],
['Chronicles','2Chr','06','019','19',"Have respect therefore to the prayer of thy servant, and to his supplication, O LORD my God, to hearken unto the cry and the prayer which thy servant prayeth before thee:"],
['Chronicles','2Chr','06','020','20',"That thine eyes may be open upon this house day and night, upon the place whereof thou hast said that thou wouldest put thy name there; to hearken unto the prayer which thy servant prayeth toward this place."],
['Chronicles','2Chr','06','021','21',"Hearken therefore unto the supplications of thy servant, and of thy people Israel, which they shall make toward this place: hear thou from thy dwelling place, \emph{even} from heaven; and when thou hearest, forgive."],
['Chronicles','2Chr','06','022','22',"If a man sin against his neighbour, and an oath be laid upon him to make him swear, and the oath come before thine altar in this house;"],
['Chronicles','2Chr','06','023','23',"Then hear thou from heaven, and do, and judge thy servants, by requiting the wicked, by recompensing his way upon his own head; and by justifying the righteous, by giving him according to his righteousness."],
['Chronicles','2Chr','06','024','24',"And if thy people Israel be put to the worse before the enemy, because they have sinned against thee; and shall return and confess thy name, and pray and make supplication before thee in this house;"],
['Chronicles','2Chr','06','025','25',"Then hear thou from the heavens, and forgive the sin of thy people Israel, and bring them again unto the land which thou gavest to them and to their fathers."],
['Chronicles','2Chr','06','026','26',"When the heaven is shut up, and there is no rain, because they have sinned against thee; \emph{yet} if they pray toward this place, and confess thy name, and turn from their sin, when thou dost afflict them;"],
['Chronicles','2Chr','06','027','27',"Then hear thou from heaven, and forgive the sin of thy servants, and of thy people Israel, when thou hast taught them the good way, wherein they should walk; and send rain upon thy land, which thou hast given unto thy people for an inheritance."],
['Chronicles','2Chr','06','028','28',"If there be dearth in the land, if there be pestilence, if there be blasting, or mildew, locusts, or caterpillers; if their enemies besiege them in the cities of their land; whatsoever sore or whatsoever sickness \emph{there} \emph{be}:"],
['Chronicles','2Chr','06','029','29',"\emph{Then} what prayer \emph{or} what supplication soever shall be made of any man, or of all thy people Israel, when every one shall know his own sore and his own grief, and shall spread forth his hands in this house:"],
['Chronicles','2Chr','06','030','30',"Then hear thou from heaven thy dwelling place, and forgive, and render unto every man according unto all his ways, whose heart thou knowest; (for thou only knowest the hearts of the children of men:)"],
['Chronicles','2Chr','06','031','31',"That they may fear thee, to walk in thy ways, so long as they live in the land which thou gavest unto our fathers."],
['Chronicles','2Chr','06','032','32',"Moreover concerning the stranger, which is not of thy people Israel, but is come from a far country for thy great name's sake, and thy mighty hand, and thy stretched out arm; if they come and pray in this house;"],
['Chronicles','2Chr','06','033','33',"Then hear thou from the heavens, \emph{even} from thy dwelling place, and do according to all that the stranger calleth to thee for; that all people of the earth may know thy name, and fear thee, as \emph{doth} thy people Israel, and may know that this house which I have built is called by thy name."],
['Chronicles','2Chr','06','034','34',"If thy people go out to war against their enemies by the way that thou shalt send them, and they pray unto thee toward this city which thou hast chosen, and the house which I have built for thy name;"],
['Chronicles','2Chr','06','035','35',"Then hear thou from the heavens their prayer and their supplication, and maintain their cause."],
['Chronicles','2Chr','06','036','36',"If they sin against thee, (for \emph{there} \emph{is} no man which sinneth not,) and thou be angry with them, and deliver them over before \emph{their} enemies, and they carry them away captives unto a land far off or near;"],
['Chronicles','2Chr','06','037','37',"Yet \emph{if} they bethink themselves in the land whither they are carried captive, and turn and pray unto thee in the land of their captivity, saying, We have sinned, we have done amiss, and have dealt wickedly;"],
['Chronicles','2Chr','06','038','38',"If they return to thee with all their heart and with all their soul in the land of their captivity, whither they have carried them captives, and pray toward their land, which thou gavest unto their fathers, and \emph{toward} the city which thou hast chosen, and toward the house which I have built for thy name:"],
['Chronicles','2Chr','06','039','39',"Then hear thou from the heavens, \emph{even} from thy dwelling place, their prayer and their supplications, and maintain their cause, and forgive thy people which have sinned against thee."],
['Chronicles','2Chr','06','040','40',"Now, my God, let, I beseech thee, thine eyes be open, and \emph{let} thine ears \emph{be} attent unto the prayer \emph{that} \emph{is} \emph{made} in this place."],
['Chronicles','2Chr','06','041','41',"Now therefore arise, O LORD God, into thy resting place, thou, and the ark of thy strength: let thy priests, O LORD God, be clothed with salvation, and let thy saints rejoice in goodness."],
['Chronicles','2Chr','06','042','42',"O LORD God, turn not away the face of thine anointed: remember the mercies of David thy servant."]
]

SecondChronicles07 = [
    ['Chronicles','2Chr','07','001','1',"Now when Solomon had made an end of praying, the fire came down from heaven, and consumed the burnt offering and the sacrifices; and the glory of the LORD filled the house."],
['Chronicles','2Chr','07','002','2',"And the priests could not enter into the house of the LORD, because the glory of the LORD had filled the LORD'S house."],
['Chronicles','2Chr','07','003','3',"And when all the children of Israel saw how the fire came down, and the glory of the LORD upon the house, they bowed themselves with their faces to the ground upon the pavement, and worshipped, and praised the LORD, \emph{saying}, For \emph{he} \emph{is} good; for his mercy \emph{endureth} for ever."],
['Chronicles','2Chr','07','004','4',"Then the king and all the people offered sacrifices before the LORD."],
['Chronicles','2Chr','07','005','5',"And king Solomon offered a sacrifice of twenty and two thousand oxen, and an hundred and twenty thousand sheep: so the king and all the people dedicated the house of God."],
['Chronicles','2Chr','07','006','6',"And the priests waited on their offices: the Levites also with instruments of musick of the LORD, which David the king had made to praise the LORD, because his mercy \emph{endureth} for ever, when David praised by their ministry; and the priests sounded trumpets before them, and all Israel stood."],
['Chronicles','2Chr','07','007','7',"Moreover Solomon hallowed the middle of the court that \emph{was} before the house of the LORD: for there he offered burnt offerings, and the fat of the peace offerings, because the brasen altar which Solomon had made was not able to receive the burnt offerings, and the meat offerings, and the fat."],
['Chronicles','2Chr','07','008','8',"Also at the same time Solomon kept the feast seven days, and all Israel with him, a very great congregation, from the entering in of Hamath unto the river of Egypt."],
['Chronicles','2Chr','07','009','9',"And in the eighth day they made a solemn assembly: for they kept the dedication of the altar seven days, and the feast seven days."],
['Chronicles','2Chr','07','010','10',"And on the three and twentieth day of the seventh month he sent the people away into their tents, glad and merry in heart for the goodness that the LORD had shewed unto David, and to Solomon, and to Israel his people."],
['Chronicles','2Chr','07','011','11',"Thus Solomon finished the house of the LORD, and the king's house: and all that came into Solomon's heart to make in the house of the LORD, and in his own house, he prosperously effected."],
['Chronicles','2Chr','07','012','12',"And the LORD appeared to Solomon by night, and said unto him, I have heard thy prayer, and have chosen this place to myself for an house of sacrifice."],
['Chronicles','2Chr','07','013','13',"If I shut up heaven that there be no rain, or if I command the locusts to devour the land, or if I send pestilence among my people;"],
['Chronicles','2Chr','07','014','14',"If my people, which are called by my name, shall humble themselves, and pray, and seek my face, and turn from their wicked ways; then will I hear from heaven, and will forgive their sin, and will heal their land."],
['Chronicles','2Chr','07','015','15',"Now mine eyes shall be open, and mine ears attent unto the prayer \emph{that} \emph{is} \emph{made} in this place."],
['Chronicles','2Chr','07','016','16',"For now have I chosen and sanctified this house, that my name may be there for ever: and mine eyes and mine heart shall be there perpetually."],
['Chronicles','2Chr','07','017','17',"And as for thee, if thou wilt walk before me, as David thy father walked, and do according to all that I have commanded thee, and shalt observe my statutes and my judgments;"],
['Chronicles','2Chr','07','018','18',"Then will I stablish the throne of thy kingdom, according as I have covenanted with David thy father, saying, There shall not fail thee a man \emph{to} \emph{be} ruler in Israel."],
['Chronicles','2Chr','07','019','19',"But if ye turn away, and forsake my statutes and my commandments, which I have set before you, and shall go and serve other gods, and worship them;"],
['Chronicles','2Chr','07','020','20',"Then will I pluck them up by the roots out of my land which I have given them; and this house, which I have sanctified for my name, will I cast out of my sight, and will make it \emph{to} \emph{be} a proverb and a byword among all nations."],
['Chronicles','2Chr','07','021','21',"And this house, which is high, shall be an astonishment to every one that passeth by it; so that he shall say, Why hath the LORD done thus unto this land, and unto this house?"],
['Chronicles','2Chr','07','022','22',"And it shall be answered, Because they forsook the LORD God of their fathers, which brought them forth out of the land of Egypt, and laid hold on other gods, and worshipped them, and served them: therefore hath he brought all this evil upon them."]
]

SecondChronicles08 = [
    ['Chronicles','2Chr','08','001','1',"And it came to pass at the end of twenty years, wherein Solomon had built the house of the LORD, and his own house,"],
['Chronicles','2Chr','08','002','2',"That the cities which Huram had restored to Solomon, Solomon built them, and caused the children of Israel to dwell there."],
['Chronicles','2Chr','08','003','3',"And Solomon went to Hamath-zobah, and prevailed against it."],
['Chronicles','2Chr','08','004','4',"And he built Tadmor in the wilderness, and all the store cities, which he built in Hamath."],
['Chronicles','2Chr','08','005','5',"Also he built Beth-horon the upper, and Beth-horon the nether, fenced cities, with walls, gates, and bars;"],
['Chronicles','2Chr','08','006','6',"And Baalath, and all the store cities that Solomon had, and all the chariot cities, and the cities of the horsemen, and all that Solomon desired to build in Jerusalem, and in Lebanon, and throughout all the land of his dominion."],
['Chronicles','2Chr','08','007','7',"\emph{As} \emph{for} all the people \emph{that} \emph{were} left of the Hittites, and the Amorites, and the Perizzites, and the Hivites, and the Jebusites, which \emph{were} not of Israel,"],
['Chronicles','2Chr','08','008','8',"\emph{But} of their children, who were left after them in the land, whom the children of Israel consumed not, them did Solomon make to pay tribute until this day."],
['Chronicles','2Chr','08','009','9',"But of the children of Israel did Solomon make no servants for his work; but they \emph{were} men of war, and chief of his captains, and captains of his chariots and horsemen."],
['Chronicles','2Chr','08','010','10',"And these \emph{were} the chief of king Solomon's officers, \emph{even} two hundred and fifty, that bare rule over the people."],
['Chronicles','2Chr','08','011','11',"And Solomon brought up the daughter of Pharaoh out of the city of David unto the house that he had built for her: for he said, My wife shall not dwell in the house of David king of Israel, because \emph{the} \emph{places} \emph{are} holy, whereunto the ark of the LORD hath come."],
['Chronicles','2Chr','08','012','12',"Then Solomon offered burnt offerings unto the LORD on the altar of the LORD, which he had built before the porch,"],
['Chronicles','2Chr','08','013','13',"Even after a certain rate every day, offering according to the commandment of Moses, on the sabbaths, and on the new moons, and on the solemn feasts, three times in the year, \emph{even} in the feast of unleavened bread, and in the feast of weeks, and in the feast of tabernacles."],
['Chronicles','2Chr','08','014','14',"And he appointed, according to the order of David his father, the courses of the priests to their service, and the Levites to their charges, to praise and minister before the priests, as the duty of every day required: the porters also by their courses at every gate: for so had David the man of God commanded."],
['Chronicles','2Chr','08','015','15',"And they departed not from the commandment of the king unto the priests and Levites concerning any matter, or concerning the treasures."],
['Chronicles','2Chr','08','016','16',"Now all the work of Solomon was prepared unto the day of the foundation of the house of the LORD, and until it was finished. \emph{So} the house of the LORD was perfected."],
['Chronicles','2Chr','08','017','17',"Then went Solomon to Ezion-geber, and to Eloth, at the sea side in the land of Edom."],
['Chronicles','2Chr','08','018','18',"And Huram sent him by the hands of his servants ships, and servants that had knowledge of the sea; and they went with the servants of Solomon to Ophir, and took thence four hundred and fifty talents of gold, and brought \emph{them} to king Solomon."]
]

SecondChronicles09 = [
    ['Chronicles','2Chr','09','001','1',"And when the queen of Sheba heard of the fame of Solomon, she came to prove Solomon with hard questions at Jerusalem, with a very great company, and camels that bare spices, and gold in abundance, and precious stones: and when she was come to Solomon, she communed with him of all that was in her heart."],
['Chronicles','2Chr','09','002','2',"And Solomon told her all her questions: and there was nothing hid from Solomon which he told her not."],
['Chronicles','2Chr','09','003','3',"And when the queen of Sheba had seen the wisdom of Solomon, and the house that he had built,"],
['Chronicles','2Chr','09','004','4',"And the meat of his table, and the sitting of his servants, and the attendance of his ministers, and their apparel; his cupbearers also, and their apparel; and his ascent by which he went up into the house of the LORD; there was no more spirit in her."],
['Chronicles','2Chr','09','005','5',"And she said to the king, \emph{It} \emph{was} a true report which I heard in mine own land of thine acts, and of thy wisdom:"],
['Chronicles','2Chr','09','006','6',"Howbeit I believed not their words, until I came, and mine eyes had seen \emph{it}: and, behold, the one half of the greatness of thy wisdom was not told me: \emph{for} thou exceedest the fame that I heard."],
['Chronicles','2Chr','09','007','7',"Happy \emph{are} thy men, and happy \emph{are} these thy servants, which stand continually before thee, and hear thy wisdom."],
['Chronicles','2Chr','09','008','8',"Blessed be the LORD thy God, which delighted in thee to set thee on his throne, \emph{to} \emph{be} king for the LORD thy God: because thy God loved Israel, to establish them for ever, therefore made he thee king over them, to do judgment and justice."],
['Chronicles','2Chr','09','009','9',"And she gave the king an hundred and twenty talents of gold, and of spices great abundance, and precious stones: neither was there any such spice as the queen of Sheba gave king Solomon."],
['Chronicles','2Chr','09','010','10',"And the servants also of Huram, and the servants of Solomon, which brought gold from Ophir, brought algum trees and precious stones."],
['Chronicles','2Chr','09','011','11',"And the king made \emph{of} the algum trees terraces to the house of the LORD, and to the king's palace, and harps and psalteries for singers: and there were none such seen before in the land of Judah."],
['Chronicles','2Chr','09','012','12',"And king Solomon gave to the queen of Sheba all her desire, whatsoever she asked, beside \emph{that} which she had brought unto the king. So she turned, and went away to her own land, she and her servants."],
['Chronicles','2Chr','09','013','13',"Now the weight of gold that came to Solomon in one year was six hundred and threescore and six talents of gold;"],
['Chronicles','2Chr','09','014','14',"Beside \emph{that} \emph{which} chapmen and merchants brought. And all the kings of Arabia and governors of the country brought gold and silver to Solomon."],
['Chronicles','2Chr','09','015','15',"And king Solomon made two hundred targets \emph{of} beaten gold: six hundred \emph{shekels} of beaten gold went to one target."],
['Chronicles','2Chr','09','016','16',"And three hundred shields \emph{made} \emph{he} \emph{of} beaten gold: three hundred \emph{shekels} of gold went to one shield. And the king put them in the house of the forest of Lebanon."],
['Chronicles','2Chr','09','017','17',"Moreover the king made a great throne of ivory, and overlaid it with pure gold."],
['Chronicles','2Chr','09','018','18',"And \emph{there} \emph{were} six steps to the throne, with a footstool of gold, \emph{which} \emph{were} fastened to the throne, and stays on each side of the sitting place, and two lions standing by the stays:"],
['Chronicles','2Chr','09','019','19',"And twelve lions stood there on the one side and on the other upon the six steps. There was not the like made in any kingdom."],
['Chronicles','2Chr','09','020','20',"And all the drinking vessels of king Solomon \emph{were} \emph{of} gold, and all the vessels of the house of the forest of Lebanon \emph{were} \emph{of} pure gold: none \emph{were} \emph{of} silver; it was \emph{not} any thing accounted of in the days of Solomon."],
['Chronicles','2Chr','09','021','21',"For the king's ships went to Tarshish with the servants of Huram: every three years once came the ships of Tarshish bringing gold, and silver, ivory, and apes, and peacocks."],
['Chronicles','2Chr','09','022','22',"And king Solomon passed all the kings of the earth in riches and wisdom."],
['Chronicles','2Chr','09','023','23',"And all the kings of the earth sought the presence of Solomon, to hear his wisdom, that God had put in his heart."],
['Chronicles','2Chr','09','024','24',"And they brought every man his present, vessels of silver, and vessels of gold, and raiment, harness, and spices, horses, and mules, a rate year by year."],
['Chronicles','2Chr','09','025','25',"And Solomon had four thousand stalls for horses and chariots, and twelve thousand horsemen; whom he bestowed in the chariot cities, and with the king at Jerusalem."],
['Chronicles','2Chr','09','026','26',"And he reigned over all the kings from the river even unto the land of the Philistines, and to the border of Egypt."],
['Chronicles','2Chr','09','027','27',"And the king made silver in Jerusalem as stones, and cedar trees made he as the sycomore trees that \emph{are} in the low plains in abundance."],
['Chronicles','2Chr','09','028','28',"And they brought unto Solomon horses out of Egypt, and out of all lands."],
['Chronicles','2Chr','09','029','29',"Now the rest of the acts of Solomon, first and last, \emph{are} they not written in the book of Nathan the prophet, and in the prophecy of Ahijah the Shilonite, and in the visions of Iddo the seer against Jeroboam the son of Nebat?"],
['Chronicles','2Chr','09','030','30',"And Solomon reigned in Jerusalem over all Israel forty years."],
['Chronicles','2Chr','09','031','31',"And Solomon slept with his fathers, and he was buried in the city of David his father: and Rehoboam his son reigned in his stead."]
]

SecondChronicles10 = [
    ['2Chronicles','2Chr','10','001','1',"And Rehoboam went to Shechem: for to Shechem were all Israel come to make him king."],
['2Chronicles','2Chr','10','002','2',"And it came to pass, when Jeroboam the son of Nebat, who \emph{was} in Egypt, whither he had fled from the presence of Solomon the king, heard \emph{it}, that Jeroboam returned out of Egypt."],
['2Chronicles','2Chr','10','003','3',"And they sent and called him. So Jeroboam and all Israel came and spake to Rehoboam, saying,"],
['2Chronicles','2Chr','10','004','4',"Thy father made our yoke grievous: now therefore ease thou somewhat the grievous servitude of thy father, and his heavy yoke that he put upon us, and we will serve thee."],
['2Chronicles','2Chr','10','005','5',"And he said unto them, Come again unto me after three days. And the people departed."],
['2Chronicles','2Chr','10','006','6',"And king Rehoboam took counsel with the old men that had stood before Solomon his father while he yet lived, saying, What counsel give ye \emph{me} to return answer to this people?"],
['2Chronicles','2Chr','10','007','7',"And they spake unto him, saying, If thou be kind to this people, and please them, and speak good words to them, they will be thy servants for ever."],
['2Chronicles','2Chr','10','008','8',"But he forsook the counsel which the old men gave him, and took counsel with the young men that were brought up with him, that stood before him."],
['2Chronicles','2Chr','10','009','9',"And he said unto them, What advice give ye that we may return answer to this people, which have spoken to me, saying, Ease somewhat the yoke that thy father did put upon us?"],
['2Chronicles','2Chr','10','010','10',"And the young men that were brought up with him spake unto him, saying, Thus shalt thou answer the people that spake unto thee, saying, Thy father made our yoke heavy, but make thou \emph{it} somewhat lighter for us; thus shalt thou say unto them, My little \emph{finger} shall be thicker than my father's loins."],
['2Chronicles','2Chr','10','011','11',"For whereas my father put a heavy yoke upon you, I will put more to your yoke: my father chastised you with whips, but I \emph{will} \emph{chastise} \emph{you} with scorpions."],
['2Chronicles','2Chr','10','012','12',"So Jeroboam and all the people came to Rehoboam on the third day, as the king bade, saying, Come again to me on the third day."],
['2Chronicles','2Chr','10','013','13',"And the king answered them roughly; and king Rehoboam forsook the counsel of the old men,"],
['2Chronicles','2Chr','10','014','14',"And answered them after the advice of the young men, saying, My father made your yoke heavy, but I will add thereto: my father chastised you with whips, but I \emph{will} \emph{chastise} \emph{you} with scorpions."],
['2Chronicles','2Chr','10','015','15',"So the king hearkened not unto the people: for the cause was of God, that the LORD might perform his word, which he spake by the hand of Ahijah the Shilonite to Jeroboam the son of Nebat."],
['2Chronicles','2Chr','10','016','16',"And when all Israel \emph{saw} that the king would not hearken unto them, the people answered the king, saying, What portion have we in David? and \emph{we} \emph{have} none inheritance in the son of Jesse: every man to your tents, O Israel: \emph{and} now, David, see to thine own house. So all Israel went to their tents."],
['2Chronicles','2Chr','10','017','17',"But \emph{as} \emph{for} the children of Israel that dwelt in the cities of Judah, Rehoboam reigned over them."],
['2Chronicles','2Chr','10','018','18',"Then king Rehoboam sent Hadoram that \emph{was} over the tribute; and the children of Israel stoned him with stones, that he died. But king Rehoboam made speed to get him up to \emph{his} chariot, to flee to Jerusalem."],
['2Chronicles','2Chr','10','019','19',"And Israel rebelled against the house of David unto this day."]
]

SecondChronicles11 = [
    ['2Chronicles','2Chr','11','001','1',"And when Rehoboam was come to Jerusalem, he gathered of the house of Judah and Benjamin an hundred and fourscore thousand chosen \emph{men}, which were warriors, to fight against Israel, that he might bring the kingdom again to Rehoboam."],
['2Chronicles','2Chr','11','002','2',"But the word of the LORD came to Shemaiah the man of God, saying,"],
['2Chronicles','2Chr','11','003','3',"Speak unto Rehoboam the son of Solomon, king of Judah, and to all Israel in Judah and Benjamin, saying,"],
['2Chronicles','2Chr','11','004','4',"Thus saith the LORD, Ye shall not go up, nor fight against your brethren: return every man to his house: for this thing is done of me. And they obeyed the words of the LORD, and returned from going against Jeroboam."],
['2Chronicles','2Chr','11','005','5',"And Rehoboam dwelt in Jerusalem, and built cities for defence in Judah."],
['2Chronicles','2Chr','11','006','6',"He built even Beth-lehem, and Etam, and Tekoa,"],
['2Chronicles','2Chr','11','007','7',"And Beth-zur, and Shoco, and Adullam,"],
['2Chronicles','2Chr','11','008','8',"And Gath, and Mareshah, and Ziph,"],
['2Chronicles','2Chr','11','009','9',"And Adoraim, and Lachish, and Azekah,"],
['2Chronicles','2Chr','11','010','10',"And Zorah, and Aijalon, and Hebron, which \emph{are} in Judah and in Benjamin fenced cities."],
['2Chronicles','2Chr','11','011','11',"And he fortified the strong holds, and put captains in them, and store of victual, and of oil and wine."],
['2Chronicles','2Chr','11','012','12',"And in every several city \emph{he} \emph{put} shields and spears, and made them exceeding strong, having Judah and Benjamin on his side."],
['2Chronicles','2Chr','11','013','13',"And the priests and the Levites that \emph{were} in all Israel resorted to him out of all their coasts."],
['2Chronicles','2Chr','11','014','14',"For the Levites left their suburbs and their possession, and came to Judah and Jerusalem: for Jeroboam and his sons had cast them off from executing the priest's office unto the LORD:"],
['2Chronicles','2Chr','11','015','15',"And he ordained him priests for the high places, and for the devils, and for the calves which he had made."],
['2Chronicles','2Chr','11','016','16',"And after them out of all the tribes of Israel such as set their hearts to seek the LORD God of Israel came to Jerusalem, to sacrifice unto the LORD God of their fathers."],
['2Chronicles','2Chr','11','017','17',"So they strengthened the kingdom of Judah, and made Rehoboam the son of Solomon strong, three years: for three years they walked in the way of David and Solomon."],
['2Chronicles','2Chr','11','018','18',"And Rehoboam took him Mahalath the daughter of Jerimoth the son of David to wife, \emph{and} Abihail the daughter of Eliab the son of Jesse;"],
['2Chronicles','2Chr','11','019','19',"Which bare him children; Jeush, and Shamariah, and Zaham."],
['2Chronicles','2Chr','11','020','20',"And after her he took Maachah the daughter of Absalom; which bare him Abijah, and Attai, and Ziza, and Shelomith."],
['2Chronicles','2Chr','11','021','21',"And Rehoboam loved Maachah the daughter of Absalom above all his wives and his concubines: (for he took eighteen wives, and threescore concubines; and begat twenty and eight sons, and threescore daughters.)"],
['2Chronicles','2Chr','11','022','22',"And Rehoboam made Abijah the son of Maachah the chief, \emph{to} \emph{be} ruler among his brethren: for \emph{he} \emph{thought} to make him king."],
['2Chronicles','2Chr','11','023','23',"And he dealt wisely, and dispersed of all his children throughout all the countries of Judah and Benjamin, unto every fenced city: and he gave them victual in abundance. And he desired many wives."]
]


SecondChronicles12 = [
['2Chronicles','2Chr','12','001','1',"And it came to pass, when Rehoboam had established the kingdom, and had strengthened himself, he forsook the law of the LORD, and all Israel with him."],
['2Chronicles','2Chr','12','002','2',"And it came to pass, \emph{that} in the fifth year of king Rehoboam Shishak king of Egypt came up against Jerusalem, because they had transgressed against the LORD,"],
['2Chronicles','2Chr','12','003','3',"With twelve hundred chariots, and threescore thousand horsemen: and the people \emph{were} without number that came with him out of Egypt; the Lubims, the Sukkiims, and the Ethiopians."],
['2Chronicles','2Chr','12','004','4',"And he took the fenced cities which \emph{pertained} to Judah, and came to Jerusalem."],
['2Chronicles','2Chr','12','005','5',"Then came Shemaiah the prophet to Rehoboam, and \emph{to} the princes of Judah, that were gathered together to Jerusalem because of Shishak, and said unto them, Thus saith the LORD, Ye have forsaken me, and therefore have I also left you in the hand of Shishak."],
['2Chronicles','2Chr','12','006','6',"Whereupon the princes of Israel and the king humbled themselves; and they said, The LORD \emph{is} righteous."],
['2Chronicles','2Chr','12','007','7',"And when the LORD saw that they humbled themselves, the word of the LORD came to Shemaiah, saying, They have humbled themselves; \emph{therefore} I will not destroy them, but I will grant them some deliverance; and my wrath shall not be poured out upon Jerusalem by the hand of Shishak."],
['2Chronicles','2Chr','12','008','8',"Nevertheless they shall be his servants; that they may know my service, and the service of the kingdoms of the countries."],
['2Chronicles','2Chr','12','009','9',"So Shishak king of Egypt came up against Jerusalem, and took away the treasures of the house of the LORD, and the treasures of the king's house; he took all: he carried away also the shields of gold which Solomon had made."],
['2Chronicles','2Chr','12','010','10',"Instead of which king Rehoboam made shields of brass, and committed \emph{them} to the hands of the chief of the guard, that kept the entrance of the king's house."],
['2Chronicles','2Chr','12','011','11',"And when the king entered into the house of the LORD, the guard came and fetched them, and brought them again into the guard chamber."],
['2Chronicles','2Chr','12','012','12',"And when he humbled himself, the wrath of the LORD turned from him, that he would not destroy \emph{him} altogether: and also in Judah things went well."],
['2Chronicles','2Chr','12','013','13',"So king Rehoboam strengthened himself in Jerusalem, and reigned: for Rehoboam \emph{was} one and forty years old when he began to reign, and he reigned seventeen years in Jerusalem, the city which the LORD had chosen out of all the tribes of Israel, to put his name there. And his mother's name \emph{was} Naamah an Ammonitess."],
['2Chronicles','2Chr','12','014','14',"And he did evil, because he prepared not his heart to seek the LORD."],
['2Chronicles','2Chr','12','015','15',"Now the acts of Rehoboam, first and last, \emph{are} they not written in the book of Shemaiah the prophet, and of Iddo the seer concerning genealogies? And \emph{there} \emph{were} wars between Rehoboam and Jeroboam continually."],
['2Chronicles','2Chr','12','016','16',"And Rehoboam slept with his fathers, and was buried in the city of David: and Abijah his son reigned in his stead."]
]

SecondChronicles13 = [
    ['2Chronicles','2Chr','13','001','1',"Now in the eighteenth year of king Jeroboam began Abijah to reign over Judah."],
['2Chronicles','2Chr','13','002','2',"He reigned three years in Jerusalem. His mother's name also \emph{was} Michaiah the daughter of Uriel of Gibeah. And there was war between Abijah and Jeroboam."],
['2Chronicles','2Chr','13','003','3',"And Abijah set the battle in array with an army of valiant men of war, \emph{even} four hundred thousand chosen men: Jeroboam also set the battle in array against him with eight hundred thousand chosen men, \emph{being} mighty men of valour."],
['2Chronicles','2Chr','13','004','4',"And Abijah stood up upon mount Zemaraim, which \emph{is} in mount Ephraim, and said, Hear me, thou Jeroboam, and all Israel;"],
['2Chronicles','2Chr','13','005','5',"Ought ye not to know that the LORD God of Israel gave the kingdom over Israel to David for ever, \emph{even} to him and to his sons by a covenant of salt?"],
['2Chronicles','2Chr','13','006','6',"Yet Jeroboam the son of Nebat, the servant of Solomon the son of David, is risen up, and hath rebelled against his lord."],
['2Chronicles','2Chr','13','007','7',"And there are gathered unto him vain men, the children of Belial, and have strengthened themselves against Rehoboam the son of Solomon, when Rehoboam was young and tenderhearted, and could not withstand them."],
['2Chronicles','2Chr','13','008','8',"And now ye think to withstand the kingdom of the LORD in the hand of the sons of David; and ye \emph{be} a great multitude, and \emph{there} \emph{are} with you golden calves, which Jeroboam made you for gods."],
['2Chronicles','2Chr','13','009','9',"Have ye not cast out the priests of the LORD, the sons of Aaron, and the Levites, and have made you priests after the manner of the nations of \emph{other} lands? so that whosoever cometh to consecrate himself with a young bullock and seven rams, \emph{the} \emph{same} may be a priest of \emph{them} \emph{that} \emph{are} no gods."],
['2Chronicles','2Chr','13','010','10',"But as for us, the LORD \emph{is} our God, and we have not forsaken him; and the priests, which minister unto the LORD, \emph{are} the sons of Aaron, and the Levites \emph{wait} upon \emph{their} business:"],
['2Chronicles','2Chr','13','011','11',"And they burn unto the LORD every morning and every evening burnt sacrifices and sweet incense: the shewbread also \emph{set} \emph{they} \emph{in} \emph{order} upon the pure table; and the candlestick of gold with the lamps thereof, to burn every evening: for we keep the charge of the LORD our God; but ye have forsaken him."],
['2Chronicles','2Chr','13','012','12',"And, behold, God himself \emph{is} with us for \emph{our} captain, and his priests with sounding trumpets to cry alarm against you. O children of Israel, fight ye not against the LORD God of your fathers; for ye shall not prosper."],
['2Chronicles','2Chr','13','013','13',"But Jeroboam caused an ambushment to come about behind them: so they were before Judah, and the ambushment \emph{was} behind them."],
['2Chronicles','2Chr','13','014','14',"And when Judah looked back, behold, the battle \emph{was} before and behind: and they cried unto the LORD, and the priests sounded with the trumpets."],
['2Chronicles','2Chr','13','015','15',"Then the men of Judah gave a shout: and as the men of Judah shouted, it came to pass, that God smote Jeroboam and all Israel before Abijah and Judah."],
['2Chronicles','2Chr','13','016','16',"And the children of Israel fled before Judah: and God delivered them into their hand."],
['2Chronicles','2Chr','13','017','17',"And Abijah and his people slew them with a great slaughter: so there fell down slain of Israel five hundred thousand chosen men."],
['2Chronicles','2Chr','13','018','18',"Thus the children of Israel were brought under at that time, and the children of Judah prevailed, because they relied upon the LORD God of their fathers."],
['2Chronicles','2Chr','13','019','19',"And Abijah pursued after Jeroboam, and took cities from him, Beth-el with the towns thereof, and Jeshanah with the towns thereof, and Ephrain with the towns thereof."],
['2Chronicles','2Chr','13','020','20',"Neither did Jeroboam recover strength again in the days of Abijah: and the LORD struck him, and he died."],
['2Chronicles','2Chr','13','021','21',"But Abijah waxed mighty, and married fourteen wives, and begat twenty and two sons, and sixteen daughters."],
['2Chronicles','2Chr','13','022','22',"And the rest of the acts of Abijah, and his ways, and his sayings, \emph{are} written in the story of the prophet Iddo."]
]

SecondChronicles14 = [
    ['2Chronicles','2Chr','14','001','1',"So Abijah slept with his fathers, and they buried him in the city of David: and Asa his son reigned in his stead. In his days the land was quiet ten years."],
['2Chronicles','2Chr','14','002','2',"And Asa did \emph{that} \emph{which} \emph{was} good and right in the eyes of the LORD his God:"],
['2Chronicles','2Chr','14','003','3',"For he took away the altars of the strange \emph{gods}, and the high places, and brake down the images, and cut down the groves:"],
['2Chronicles','2Chr','14','004','4',"And commanded Judah to seek the LORD God of their fathers, and to do the law and the commandment."],
['2Chronicles','2Chr','14','005','5',"Also he took away out of all the cities of Judah the high places and the images: and the kingdom was quiet before him."],
['2Chronicles','2Chr','14','006','6',"And he built fenced cities in Judah: for the land had rest, and he had no war in those years; because the LORD had given him rest."],
['2Chronicles','2Chr','14','007','7',"Therefore he said unto Judah, Let us build these cities, and make about \emph{them} walls, and towers, gates, and bars, \emph{while} the land \emph{is} yet before us; because we have sought the LORD our God, we have sought \emph{him}, and he hath given us rest on every side. So they built and prospered."],
['2Chronicles','2Chr','14','008','8',"And Asa had an army \emph{of} \emph{men} that bare targets and spears, out of Judah three hundred thousand; and out of Benjamin, that bare shields and drew bows, two hundred and fourscore thousand: all these \emph{were} mighty men of valour."],
['2Chronicles','2Chr','14','009','9',"And there came out against them Zerah the Ethiopian with an host of a thousand thousand, and three hundred chariots; and came unto Mareshah."],
['2Chronicles','2Chr','14','010','10',"Then Asa went out against him, and they set the battle in array in the valley of Zephathah at Mareshah."],
['2Chronicles','2Chr','14','011','11',"And Asa cried unto the LORD his God, and said, LORD, \emph{it} \emph{is} nothing with thee to help, whether with many, or with them that have no power: help us, O LORD our God; for we rest on thee, and in thy name we go against this multitude. O LORD, thou \emph{art} our God; let not man prevail against thee."],
['2Chronicles','2Chr','14','012','12',"So the LORD smote the Ethiopians before Asa, and before Judah; and the Ethiopians fled."],
['2Chronicles','2Chr','14','013','13',"And Asa and the people that \emph{were} with him pursued them unto Gerar: and the Ethiopians were overthrown, that they could not recover themselves; for they were destroyed before the LORD, and before his host; and they carried away very much spoil."],
['2Chronicles','2Chr','14','014','14',"And they smote all the cities round about Gerar; for the fear of the LORD came upon them: and they spoiled all the cities; for there was exceeding much spoil in them."],
['2Chronicles','2Chr','14','015','15',"They smote also the tents of cattle, and carried away sheep and camels in abundance, and returned to Jerusalem."]
]

SecondChronicles15 = [
    ['2Chronicles','2Chr','15','001','1',"And the Spirit of God came upon Azariah the son of Oded:"],
['2Chronicles','2Chr','15','002','2',"And he went out to meet Asa, and said unto him, Hear ye me, Asa, and all Judah and Benjamin; The LORD \emph{is} with you, while ye be with him; and if ye seek him, he will be found of you; but if ye forsake him, he will forsake you."],
['2Chronicles','2Chr','15','003','3',"Now for a long season Israel \emph{hath} \emph{been} without the true God, and without a teaching priest, and without law."],
['2Chronicles','2Chr','15','004','4',"But when they in their trouble did turn unto the LORD God of Israel, and sought him, he was found of them."],
['2Chronicles','2Chr','15','005','5',"And in those times \emph{there} \emph{was} no peace to him that went out, nor to him that came in, but great vexations \emph{were} upon all the inhabitants of the countries."],
['2Chronicles','2Chr','15','006','6',"And nation was destroyed of nation, and city of city: for God did vex them with all adversity."],
['2Chronicles','2Chr','15','007','7',"Be ye strong therefore, and let not your hands be weak: for your work shall be rewarded."],
['2Chronicles','2Chr','15','008','8',"And when Asa heard these words, and the prophecy of Oded the prophet, he took courage, and put away the abominable idols out of all the land of Judah and Benjamin, and out of the cities which he had taken from mount Ephraim, and renewed the altar of the LORD, that \emph{was} before the porch of the LORD."],
['2Chronicles','2Chr','15','009','9',"And he gathered all Judah and Benjamin, and the strangers with them out of Ephraim and Manasseh, and out of Simeon: for they fell to him out of Israel in abundance, when they saw that the LORD his God \emph{was} with him."],
['2Chronicles','2Chr','15','010','10',"So they gathered themselves together at Jerusalem in the third month, in the fifteenth year of the reign of Asa."],
['2Chronicles','2Chr','15','011','11',"And they offered unto the LORD the same time, of the spoil \emph{which} they had brought, seven hundred oxen and seven thousand sheep."],
['2Chronicles','2Chr','15','012','12',"And they entered into a covenant to seek the LORD God of their fathers with all their heart and with all their soul;"],
['2Chronicles','2Chr','15','013','13',"That whosoever would not seek the LORD God of Israel should be put to death, whether small or great, whether man or woman."],
['2Chronicles','2Chr','15','014','14',"And they sware unto the LORD with a loud voice, and with shouting, and with trumpets, and with cornets."],
['2Chronicles','2Chr','15','015','15',"And all Judah rejoiced at the oath: for they had sworn with all their heart, and sought him with their whole desire; and he was found of them: and the LORD gave them rest round about."],
['2Chronicles','2Chr','15','016','16',"And also \emph{concerning} Maachah the mother of Asa the king, he removed her from \emph{being} queen, because she had made an idol in a grove: and Asa cut down her idol, and stamped \emph{it}, and burnt \emph{it} at the brook Kidron."],
['2Chronicles','2Chr','15','017','17',"But the high places were not taken away out of Israel: nevertheless the heart of Asa was perfect all his days."],
['2Chronicles','2Chr','15','018','18',"And he brought into the house of God the things that his father had dedicated, and that he himself had dedicated, silver, and gold, and vessels."],
['2Chronicles','2Chr','15','019','19',"And there was no \emph{more} war unto the five and thirtieth year of the reign of Asa."]
]

SecondChronicles16 = [
    ['2Chronicles','2Chr','16','001','1',"In the six and thirtieth year of the reign of Asa Baasha king of Israel came up against Judah, and built Ramah, to the intent that he might let none go out or come in to Asa king of Judah."],
['2Chronicles','2Chr','16','002','2',"Then Asa brought out silver and gold out of the treasures of the house of the LORD and of the king's house, and sent to Ben-hadad king of Syria, that dwelt at Damascus, saying,"],
['2Chronicles','2Chr','16','003','3',"\emph{There} \emph{is} a league between me and thee, as \emph{there} \emph{was} between my father and thy father: behold, I have sent thee silver and gold; go, break thy league with Baasha king of Israel, that he may depart from me."],
['2Chronicles','2Chr','16','004','4',"And Ben-hadad hearkened unto king Asa, and sent the captains of his armies against the cities of Israel; and they smote Ijon, and Dan, and Abel-maim, and all the store cities of Naphtali."],
['2Chronicles','2Chr','16','005','5',"And it came to pass, when Baasha heard \emph{it}, that he left off building of Ramah, and let his work cease."],
['2Chronicles','2Chr','16','006','6',"Then Asa the king took all Judah; and they carried away the stones of Ramah, and the timber thereof, wherewith Baasha was building; and he built therewith Geba and Mizpah."],
['2Chronicles','2Chr','16','007','7',"And at that time Hanani the seer came to Asa king of Judah, and said unto him, Because thou hast relied on the king of Syria, and not relied on the LORD thy God, therefore is the host of the king of Syria escaped out of thine hand."],
['2Chronicles','2Chr','16','008','8',"Were not the Ethiopians and the Lubims a huge host, with very many chariots and horsemen? yet, because thou didst rely on the LORD, he delivered them into thine hand."],
['2Chronicles','2Chr','16','009','9',"For the eyes of the LORD run to and fro throughout the whole earth, to shew himself strong in the behalf of \emph{them} whose heart \emph{is} perfect toward him. Herein thou hast done foolishly: therefore from henceforth thou shalt have wars."],
['2Chronicles','2Chr','16','010','10',"Then Asa was wroth with the seer, and put him in a prison house; for \emph{he} \emph{was} in a rage with him because of this \emph{thing}. And Asa oppressed \emph{some} of the people the same time."],
['2Chronicles','2Chr','16','011','11',"And, behold, the acts of Asa, first and last, lo, they \emph{are} written in the book of the kings of Judah and Israel."],
['2Chronicles','2Chr','16','012','12',"And Asa in the thirty and ninth year of his reign was diseased in his feet, until his disease \emph{was} exceeding \emph{great}: yet in his disease he sought not to the LORD, but to the physicians."],
['2Chronicles','2Chr','16','013','13',"And Asa slept with his fathers, and died in the one and fortieth year of his reign."],
['2Chronicles','2Chr','16','014','14',"And they buried him in his own sepulchres, which he had made for himself in the city of David, and laid him in the bed which was filled with sweet odours and divers kinds \emph{of} \emph{spices} prepared by the apothecaries' art: and they made a very great burning for him."]
]

SecondChronicles17 = [
    ['2Chronicles','2Chr','17','001','1',"And Jehoshaphat his son reigned in his stead, and strengthened himself against Israel."],
['2Chronicles','2Chr','17','002','2',"And he placed forces in all the fenced cities of Judah, and set garrisons in the land of Judah, and in the cities of Ephraim, which Asa his father had taken."],
['2Chronicles','2Chr','17','003','3',"And the LORD was with Jehoshaphat, because he walked in the first ways of his father David, and sought not unto Baalim;"],
['2Chronicles','2Chr','17','004','4',"But sought to the \emph{LORD} God of his father, and walked in his commandments, and not after the doings of Israel."],
['2Chronicles','2Chr','17','005','5',"Therefore the LORD stablished the kingdom in his hand; and all Judah brought to Jehoshaphat presents; and he had riches and honour in abundance."],
['2Chronicles','2Chr','17','006','6',"And his heart was lifted up in the ways of the LORD: moreover he took away the high places and groves out of Judah."],
['2Chronicles','2Chr','17','007','7',"Also in the third year of his reign he sent to his princes, \emph{even} to Benhail, and to Obadiah, and to Zechariah, and to Nethaneel, and to Michaiah, to teach in the cities of Judah."],
['2Chronicles','2Chr','17','008','8',"And with them \emph{he} \emph{sent} Levites, \emph{even} Shemaiah, and Nethaniah, and Zebadiah, and Asahel, and Shemiramoth, and Jehonathan, and Adonijah, and Tobijah, and Tobadonijah, Levites; and with them Elishama and Jehoram, priests."],
['2Chronicles','2Chr','17','009','9',"And they taught in Judah, and \emph{had} the book of the law of the LORD with them, and went about throughout all the cities of Judah, and taught the people."],
['2Chronicles','2Chr','17','010','10',"And the fear of the LORD fell upon all the kingdoms of the lands that \emph{were} round about Judah, so that they made no war against Jehoshaphat."],
['2Chronicles','2Chr','17','011','11',"Also \emph{some} of the Philistines brought Jehoshaphat presents, and tribute silver; and the Arabians brought him flocks, seven thousand and seven hundred rams, and seven thousand and seven hundred he goats."],
['2Chronicles','2Chr','17','012','12',"And Jehoshaphat waxed great exceedingly; and he built in Judah castles, and cities of store."],
['2Chronicles','2Chr','17','013','13',"And he had much business in the cities of Judah: and the men of war, mighty men of valour, \emph{were} in Jerusalem."],
['2Chronicles','2Chr','17','014','14',"And these \emph{are} the numbers of them according to the house of their fathers: Of Judah, the captains of thousands; Adnah the chief, and with him mighty men of valour three hundred thousand."],
['2Chronicles','2Chr','17','015','15',"And next to him \emph{was} Jehohanan the captain, and with him two hundred and fourscore thousand."],
['2Chronicles','2Chr','17','016','16',"And next him \emph{was} Amasiah the son of Zichri, who willingly offered himself unto the LORD; and with him two hundred thousand mighty men of valour."],
['2Chronicles','2Chr','17','017','17',"And of Benjamin; Eliada a mighty man of valour, and with him armed men with bow and shield two hundred thousand."],
['2Chronicles','2Chr','17','018','18',"And next him \emph{was} Jehozabad, and with him an hundred and fourscore thousand ready prepared for the war."],
['2Chronicles','2Chr','17','019','19',"These waited on the king, beside \emph{those} whom the king put in the fenced cities throughout all Judah."]
]

SecondChronicles18 = [
    ['2Chronicles','2Chr','18','001','1',"Now Jehoshaphat had riches and honour in abundance, and joined affinity with Ahab."],
['2Chronicles','2Chr','18','002','2',"And after \emph{certain} years he went down to Ahab to Samaria. And Ahab killed sheep and oxen for him in abundance, and for the people that \emph{he} \emph{had} with him, and persuaded him to go up \emph{with} \emph{him} to Ramoth-gilead."],
['2Chronicles','2Chr','18','003','3',"And Ahab king of Israel said unto Jehoshaphat king of Judah, Wilt thou go with me to Ramoth-gilead? And he answered him, I \emph{am} as thou \emph{art}, and my people as thy people; and \emph{we} \emph{will} \emph{be} with thee in the war."],
['2Chronicles','2Chr','18','004','4',"And Jehoshaphat said unto the king of Israel, Enquire, I pray thee, at the word of the LORD to day."],
['2Chronicles','2Chr','18','005','5',"Therefore the king of Israel gathered together of prophets four hundred men, and said unto them, Shall we go to Ramoth-gilead to battle, or shall I forbear? And they said, Go up; for God will deliver \emph{it} into the king's hand."],
['2Chronicles','2Chr','18','006','6',"But Jehoshaphat said, \emph{Is} \emph{there} not here a prophet of the LORD besides, that we might enquire of him?"],
['2Chronicles','2Chr','18','007','7',"And the king of Israel said unto Jehoshaphat, \emph{There} \emph{is} yet one man, by whom we may enquire of the LORD: but I hate him; for he never prophesied good unto me, but always evil: the same \emph{is} Micaiah the son of Imla. And Jehoshaphat said, Let not the king say so."],
['2Chronicles','2Chr','18','008','8',"And the king of Israel called for one \emph{of} \emph{his} officers, and said, Fetch quickly Micaiah the son of Imla."],
['2Chronicles','2Chr','18','009','9',"And the king of Israel and Jehoshaphat king of Judah sat either of them on his throne, clothed in \emph{their} robes, and they sat in a void place at the entering in of the gate of Samaria; and all the prophets prophesied before them."],
['2Chronicles','2Chr','18','010','10',"And Zedekiah the son of Chenaanah had made him horns of iron, and said, Thus saith the LORD, With these thou shalt push Syria until they be consumed."],
['2Chronicles','2Chr','18','011','11',"And all the prophets prophesied so, saying, Go up to Ramoth-gilead, and prosper: for the LORD shall deliver \emph{it} into the hand of the king."],
['2Chronicles','2Chr','18','012','12',"And the messenger that went to call Micaiah spake to him, saying, Behold, the words of the prophets \emph{declare} good to the king with one assent; let thy word therefore, I pray thee, be like one of their's, and speak thou good."],
['2Chronicles','2Chr','18','013','13',"And Micaiah said, \emph{As} the LORD liveth, even what my God saith, that will I speak."],
['2Chronicles','2Chr','18','014','14',"And when he was come to the king, the king said unto him, Micaiah, shall we go to Ramoth-gilead to battle, or shall I forbear? And he said, Go ye up, and prosper, and they shall be delivered into your hand."],
['2Chronicles','2Chr','18','015','15',"And the king said to him, How many times shall I adjure thee that thou say nothing but the truth to me in the name of the LORD?"],
['2Chronicles','2Chr','18','016','16',"Then he said, I did see all Israel scattered upon the mountains, as sheep that have no shepherd: and the LORD said, These have no master; let them return \emph{therefore} every man to his house in peace."],
['2Chronicles','2Chr','18','017','17',"And the king of Israel said to Jehoshaphat, Did I not tell thee \emph{that} he would not prophesy good unto me, but evil?"],
['2Chronicles','2Chr','18','018','18',"Again he said, Therefore hear the word of the LORD; I saw the LORD sitting upon his throne, and all the host of heaven standing on his right hand and \emph{on} his left."],
['2Chronicles','2Chr','18','019','19',"And the LORD said, Who shall entice Ahab king of Israel, that he may go up and fall at Ramoth-gilead? And one spake saying after this manner, and another saying after that manner."],
['2Chronicles','2Chr','18','020','20',"Then there came out a spirit, and stood before the LORD, and said, I will entice him. And the LORD said unto him, Wherewith?"],
['2Chronicles','2Chr','18','021','21',"And he said, I will go out, and be a lying spirit in the mouth of all his prophets. And \emph{the} \emph{LORD} said, Thou shalt entice \emph{him}, and thou shalt also prevail: go out, and do \emph{even} so."],
['2Chronicles','2Chr','18','022','22',"Now therefore, behold, the LORD hath put a lying spirit in the mouth of these thy prophets, and the LORD hath spoken evil against thee."],
['2Chronicles','2Chr','18','023','23',"Then Zedekiah the son of Chenaanah came near, and smote Micaiah upon the cheek, and said, Which way went the Spirit of the LORD from me to speak unto thee?"],
['2Chronicles','2Chr','18','024','24',"And Micaiah said, Behold, thou shalt see on that day when thou shalt go into an inner chamber to hide thyself."],
['2Chronicles','2Chr','18','025','25',"Then the king of Israel said, Take ye Micaiah, and carry him back to Amon the governor of the city, and to Joash the king's son;"],
['2Chronicles','2Chr','18','026','26',"And say, Thus saith the king, Put this \emph{fellow} in the prison, and feed him with bread of affliction and with water of affliction, until I return in peace."],
['2Chronicles','2Chr','18','027','27',"And Micaiah said, If thou certainly return in peace, \emph{then} hath not the LORD spoken by me. And he said, Hearken, all ye people."],
['2Chronicles','2Chr','18','028','28',"So the king of Israel and Jehoshaphat the king of Judah went up to Ramoth-gilead."],
['2Chronicles','2Chr','18','029','29',"And the king of Israel said unto Jehoshaphat, I will disguise myself, and will go to the battle; but put thou on thy robes. So the king of Israel disguised himself; and they went to the battle."],
['2Chronicles','2Chr','18','030','30',"Now the king of Syria had commanded the captains of the chariots that \emph{were} with him, saying, Fight ye not with small or great, save only with the king of Israel."],
['2Chronicles','2Chr','18','031','31',"And it came to pass, when the captains of the chariots saw Jehoshaphat, that they said, It \emph{is} the king of Israel. Therefore they compassed about him to fight: but Jehoshaphat cried out, and the LORD helped him; and God moved them \emph{to} \emph{depart} from him."],
['2Chronicles','2Chr','18','032','32',"For it came to pass, that, when the captains of the chariots perceived that it was not the king of Israel, they turned back again from pursuing him."],
['2Chronicles','2Chr','18','033','33',"And a \emph{certain} man drew a bow at a venture, and smote the king of Israel between the joints of the harness: therefore he said to his chariot man, Turn thine hand, that thou mayest carry me out of the host; for I am wounded."],
['2Chronicles','2Chr','18','034','34',"And the battle increased that day: howbeit the king of Israel stayed \emph{himself} up in \emph{his} chariot against the Syrians until the even: and about the time of the sun going down he died."]
]

SecondChronicles19 = [
    ['2Chronicles','2Chr','19','001','1',"And Jehoshaphat the king of Judah returned to his house in peace to Jerusalem."],
['2Chronicles','2Chr','19','002','2',"And Jehu the son of Hanani the seer went out to meet him, and said to king Jehoshaphat, Shouldest thou help the ungodly, and love them that hate the LORD? therefore \emph{is} wrath upon thee from before the LORD."],
['2Chronicles','2Chr','19','003','3',"Nevertheless there are good things found in thee, in that thou hast taken away the groves out of the land, and hast prepared thine heart to seek God."],
['2Chronicles','2Chr','19','004','4',"And Jehoshaphat dwelt at Jerusalem: and he went out again through the people from Beer-sheba to mount Ephraim, and brought them back unto the LORD God of their fathers."],
['2Chronicles','2Chr','19','005','5',"And he set judges in the land throughout all the fenced cities of Judah, city by city,"],
['2Chronicles','2Chr','19','006','6',"And said to the judges, Take heed what ye do: for ye judge not for man, but for the LORD, who \emph{is} with you in the judgment."],
['2Chronicles','2Chr','19','007','7',"Wherefore now let the fear of the LORD be upon you; take heed and do \emph{it}: for \emph{there} \emph{is} no iniquity with the LORD our God, nor respect of persons, nor taking of gifts."],
['2Chronicles','2Chr','19','008','8',"Moreover in Jerusalem did Jehoshaphat set of the Levites, and \emph{of} the priests, and of the chief of the fathers of Israel, for the judgment of the LORD, and for controversies, when they returned to Jerusalem."],
['2Chronicles','2Chr','19','009','9',"And he charged them, saying, Thus shall ye do in the fear of the LORD, faithfully, and with a perfect heart."],
['2Chronicles','2Chr','19','010','10',"And what cause soever shall come to you of your brethren that dwell in their cities, between blood and blood, between law and commandment, statutes and judgments, ye shall even warn them that they trespass not against the LORD, and \emph{so} wrath come upon you, and upon your brethren: this do, and ye shall not trespass."],
['2Chronicles','2Chr','19','011','11',"And, behold, Amariah the chief priest \emph{is} over you in all matters of the LORD; and Zebadiah the son of Ishmael, the ruler of the house of Judah, for all the king's matters: also the Levites \emph{shall} \emph{be} officers before you. Deal courageously, and the LORD shall be with the good."]
]

SecondChronicles20 = [
    ['2Chronicles','2Chr','20','001','1',"It came to pass after this also, \emph{that} the children of Moab, and the children of Ammon, and with them \emph{other} beside the Ammonites, came against Jehoshaphat to battle."],
['2Chronicles','2Chr','20','002','2',"Then there came some that told Jehoshaphat, saying, There cometh a great multitude against thee from beyond the sea on this side Syria; and, behold, they \emph{be} in Hazazon-tamar, which \emph{is} En-gedi."],
['2Chronicles','2Chr','20','003','3',"And Jehoshaphat feared, and set himself to seek the LORD, and proclaimed a fast throughout all Judah."],
['2Chronicles','2Chr','20','004','4',"And Judah gathered themselves together, to ask \emph{help} of the LORD: even out of all the cities of Judah they came to seek the LORD."],
['2Chronicles','2Chr','20','005','5',"And Jehoshaphat stood in the congregation of Judah and Jerusalem, in the house of the LORD, before the new court,"],
['2Chronicles','2Chr','20','006','6',"And said, O LORD God of our fathers, \emph{art} not thou God in heaven? and rulest \emph{not} thou over all the kingdoms of the heathen? and in thine hand \emph{is} \emph{there} \emph{not} power and might, so that none is able to withstand thee?"],
['2Chronicles','2Chr','20','007','7',"\emph{Art} not thou our God, \emph{who} didst drive out the inhabitants of this land before thy people Israel, and gavest it to the seed of Abraham thy friend for ever?"],
['2Chronicles','2Chr','20','008','8',"And they dwelt therein, and have built thee a sanctuary therein for thy name, saying,"],
['2Chronicles','2Chr','20','009','9',"If, \emph{when} evil cometh upon us, \emph{as} the sword, judgment, or pestilence, or famine, we stand before this house, and in thy presence, (for thy name \emph{is} in this house,) and cry unto thee in our affliction, then thou wilt hear and help."],
['2Chronicles','2Chr','20','010','10',"And now, behold, the children of Ammon and Moab and mount Seir, whom thou wouldest not let Israel invade, when they came out of the land of Egypt, but they turned from them, and destroyed them not;"],
['2Chronicles','2Chr','20','011','11',"Behold, \emph{I} \emph{say,} \emph{how} they reward us, to come to cast us out of thy possession, which thou hast given us to inherit."],
['2Chronicles','2Chr','20','012','12',"O our God, wilt thou not judge them? for we have no might against this great company that cometh against us; neither know we what to do: but our eyes \emph{are} upon thee."],
['2Chronicles','2Chr','20','013','13',"And all Judah stood before the LORD, with their little ones, their wives, and their children."],
['2Chronicles','2Chr','20','014','14',"Then upon Jahaziel the son of Zechariah, the son of Benaiah, the son of Jeiel, the son of Mattaniah, a Levite of the sons of Asaph, came the Spirit of the LORD in the midst of the congregation;"],
['2Chronicles','2Chr','20','015','15',"And he said, Hearken ye, all Judah, and ye inhabitants of Jerusalem, and thou king Jehoshaphat, Thus saith the LORD unto you, Be not afraid nor dismayed by reason of this great multitude; for the battle \emph{is} not your's, but God's."],
['2Chronicles','2Chr','20','016','16',"To morrow go ye down against them: behold, they come up by the cliff of Ziz; and ye shall find them at the end of the brook, before the wilderness of Jeruel."],
['2Chronicles','2Chr','20','017','17',"Ye shall not \emph{need} to fight in this \emph{battle}: set yourselves, stand ye \emph{still}, and see the salvation of the LORD with you, O Judah and Jerusalem: fear not, nor be dismayed; to morrow go out against them: for the LORD \emph{will} \emph{be} with you."],
['2Chronicles','2Chr','20','018','18',"And Jehoshaphat bowed his head with \emph{his} face to the ground: and all Judah and the inhabitants of Jerusalem fell before the LORD, worshipping the LORD."],
['2Chronicles','2Chr','20','019','19',"And the Levites, of the children of the Kohathites, and of the children of the Korhites, stood up to praise the LORD God of Israel with a loud voice on high."],
['2Chronicles','2Chr','20','020','20',"And they rose early in the morning, and went forth into the wilderness of Tekoa: and as they went forth, Jehoshaphat stood and said, Hear me, O Judah, and ye inhabitants of Jerusalem; Believe in the LORD your God, so shall ye be established; believe his prophets, so shall ye prosper."],
['2Chronicles','2Chr','20','021','21',"And when he had consulted with the people, he appointed singers unto the LORD, and that should praise the beauty of holiness, as they went out before the army, and to say, Praise the LORD; for his mercy \emph{endureth} for ever."],
['2Chronicles','2Chr','20','022','22',"And when they began to sing and to praise, the LORD set ambushments against the children of Ammon, Moab, and mount Seir, which were come against Judah; and they were smitten."],
['2Chronicles','2Chr','20','023','23',"For the children of Ammon and Moab stood up against the inhabitants of mount Seir, utterly to slay and destroy \emph{them}: and when they had made an end of the inhabitants of Seir, every one helped to destroy another."],
['2Chronicles','2Chr','20','024','24',"And when Judah came toward the watch tower in the wilderness, they looked unto the multitude, and, behold, they \emph{were} dead bodies fallen to the earth, and none escaped."],
['2Chronicles','2Chr','20','025','25',"And when Jehoshaphat and his people came to take away the spoil of them, they found among them in abundance both riches with the dead bodies, and precious jewels, which they stripped off for themselves, more than they could carry away: and they were three days in gathering of the spoil, it was so much."],
['2Chronicles','2Chr','20','026','26',"And on the fourth day they assembled themselves in the valley of Berachah; for there they blessed the LORD: therefore the name of the same place was called, The valley of Berachah, unto this day."],
['2Chronicles','2Chr','20','027','27',"Then they returned, every man of Judah and Jerusalem, and Jehoshaphat in the forefront of them, to go again to Jerusalem with joy; for the LORD had made them to rejoice over their enemies."],
['2Chronicles','2Chr','20','028','28',"And they came to Jerusalem with psalteries and harps and trumpets unto the house of the LORD."],
['2Chronicles','2Chr','20','029','29',"And the fear of God was on all the kingdoms of \emph{those} countries, when they had heard that the LORD fought against the enemies of Israel."],
['2Chronicles','2Chr','20','030','30',"So the realm of Jehoshaphat was quiet: for his God gave him rest round about."],
['2Chronicles','2Chr','20','031','31',"And Jehoshaphat reigned over Judah: \emph{he} \emph{was} thirty and five years old when he began to reign, and he reigned twenty and five years in Jerusalem. And his mother's name \emph{was} Azubah the daughter of Shilhi."],
['2Chronicles','2Chr','20','032','32',"And he walked in the way of Asa his father, and departed not from it, doing \emph{that} \emph{which} \emph{was} right in the sight of the LORD."],
['2Chronicles','2Chr','20','033','33',"Howbeit the high places were not taken away: for as yet the people had not prepared their hearts unto the God of their fathers."],
['2Chronicles','2Chr','20','034','34',"Now the rest of the acts of Jehoshaphat, first and last, behold, they \emph{are} written in the book of Jehu the son of Hanani, who \emph{is} mentioned in the book of the kings of Israel."],
['2Chronicles','2Chr','20','035','35',"And after this did Jehoshaphat king of Judah join himself with Ahaziah king of Israel, who did very wickedly:"],
['2Chronicles','2Chr','20','036','36',"And he joined himself with him to make ships to go to Tarshish: and they made the ships in Ezion-geber."],
['2Chronicles','2Chr','20','037','37',"Then Eliezer the son of Dodavah of Mareshah prophesied against Jehoshaphat, saying, Because thou hast joined thyself with Ahaziah, the LORD hath broken thy works. And the ships were broken, that they were not able to go to Tarshish."]
]

SecondChronicles21 = [
    ['2Chronicles','2Chr','21','001','1',"Now Jehoshaphat slept with his fathers, and was buried with his fathers in the city of David. And Jehoram his son reigned in his stead."],
['2Chronicles','2Chr','21','002','2',"And he had brethren the sons of Jehoshaphat, Azariah, and Jehiel, and Zechariah, and Azariah, and Michael, and Shephatiah: all these \emph{were} the sons of Jehoshaphat king of Israel."],
['2Chronicles','2Chr','21','003','3',"And their father gave them great gifts of silver, and of gold, and of precious things, with fenced cities in Judah: but the kingdom gave he to Jehoram; because he \emph{was} the firstborn."],
['2Chronicles','2Chr','21','004','4',"Now when Jehoram was risen up to the kingdom of his father, he strengthened himself, and slew all his brethren with the sword, and \emph{divers} also of the princes of Israel."],
['2Chronicles','2Chr','21','005','5',"Jehoram \emph{was} thirty and two years old when he began to reign, and he reigned eight years in Jerusalem."],
['2Chronicles','2Chr','21','006','6',"And he walked in the way of the kings of Israel, like as did the house of Ahab: for he had the daughter of Ahab to wife: and he wrought \emph{that} \emph{which} \emph{was} evil in the eyes of the LORD."],
['2Chronicles','2Chr','21','007','7',"Howbeit the LORD would not destroy the house of David, because of the covenant that he had made with David, and as he promised to give a light to him and to his sons for ever."],
['2Chronicles','2Chr','21','008','8',"In his days the Edomites revolted from under the dominion of Judah, and made themselves a king."],
['2Chronicles','2Chr','21','009','9',"Then Jehoram went forth with his princes, and all his chariots with him: and he rose up by night, and smote the Edomites which compassed him in, and the captains of the chariots."],
['2Chronicles','2Chr','21','010','10',"So the Edomites revolted from under the hand of Judah unto this day. The same time \emph{also} did Libnah revolt from under his hand; because he had forsaken the LORD God of his fathers."],
['2Chronicles','2Chr','21','011','11',"Moreover he made high places in the mountains of Judah, and caused the inhabitants of Jerusalem to commit fornication, and compelled Judah \emph{thereto}."],
['2Chronicles','2Chr','21','012','12',"And there came a writing to him from Elijah the prophet, saying, Thus saith the LORD God of David thy father, Because thou hast not walked in the ways of Jehoshaphat thy father, nor in the ways of Asa king of Judah,"],
['2Chronicles','2Chr','21','013','13',"But hast walked in the way of the kings of Israel, and hast made Judah and the inhabitants of Jerusalem to go a whoring, like to the whoredoms of the house of Ahab, and also hast slain thy brethren of thy father's house, \emph{which} \emph{were} better than thyself:"],
['2Chronicles','2Chr','21','014','14',"Behold, with a great plague will the LORD smite thy people, and thy children, and thy wives, and all thy goods:"],
['2Chronicles','2Chr','21','015','15',"And thou \emph{shalt} \emph{have} great sickness by disease of thy bowels, until thy bowels fall out by reason of the sickness day by day."],
['2Chronicles','2Chr','21','016','16',"Moreover the LORD stirred up against Jehoram the spirit of the Philistines, and of the Arabians, that \emph{were} near the Ethiopians:"],
['2Chronicles','2Chr','21','017','17',"And they came up into Judah, and brake into it, and carried away all the substance that was found in the king's house, and his sons also, and his wives; so that there was never a son left him, save Jehoahaz, the youngest of his sons."],
['2Chronicles','2Chr','21','018','18',"And after all this the LORD smote him in his bowels with an incurable disease."],
    ['2Chronicles','2Chr','21','019','19',"And it came to pass, that in process of time, after the end of two years, his bowels fell out by reason of his sickness: so he died of sore diseases. And his people made no burning for him, like the burning of his fathers."],
    ['2Chronicles','2Chr','21','020','20',"Thirty and two years old was he when he began to reign, and he reigned in Jerusalem eight years, and departed without being desired. Howbeit they buried him in the city of David, but not in the sepulchres of the kings."]
]

SecondChronicles22 = [
    ['2Chronicles','2Chr','22','001','1',"And the inhabitants of Jerusalem made Ahaziah his youngest son king in his stead: for the band of men that came with the Arabians to the camp had slain all the eldest. So Ahaziah the son of Jehoram king of Judah reigned."],
['2Chronicles','2Chr','22','002','2',"Forty and two years old \emph{was} Ahaziah when he began to reign, and he reigned one year in Jerusalem. His mother's name also \emph{was} Athaliah the daughter of Omri."],
['2Chronicles','2Chr','22','003','3',"He also walked in the ways of the house of Ahab: for his mother was his counsellor to do wickedly."],
['2Chronicles','2Chr','22','004','4',"Wherefore he did evil in the sight of the LORD like the house of Ahab: for they were his counsellors after the death of his father to his destruction."],
['2Chronicles','2Chr','22','005','5',"He walked also after their counsel, and went with Jehoram the son of Ahab king of Israel to war against Hazael king of Syria at Ramoth-gilead: and the Syrians smote Joram."],
['2Chronicles','2Chr','22','006','6',"And he returned to be healed in Jezreel because of the wounds which were given him at Ramah, when he fought with Hazael king of Syria. And Azariah the son of Jehoram king of Judah went down to see Jehoram the son of Ahab at Jezreel, because he was sick."],
['2Chronicles','2Chr','22','007','7',"And the destruction of Ahaziah was of God by coming to Joram: for when he was come, he went out with Jehoram against Jehu the son of Nimshi, whom the LORD had anointed to cut off the house of Ahab."],
['2Chronicles','2Chr','22','008','8',"And it came to pass, that, when Jehu was executing judgment upon the house of Ahab, and found the princes of Judah, and the sons of the brethren of Ahaziah, that ministered to Ahaziah, he slew them."],
['2Chronicles','2Chr','22','009','9',"And he sought Ahaziah: and they caught him, (for he was hid in Samaria,) and brought him to Jehu: and when they had slain him, they buried him: Because, said they, he \emph{is} the son of Jehoshaphat, who sought the LORD with all his heart. So the house of Ahaziah had no power to keep still the kingdom."],
['2Chronicles','2Chr','22','010','10',"But when Athaliah the mother of Ahaziah saw that her son was dead, she arose and destroyed all the seed royal of the house of Judah."],
['2Chronicles','2Chr','22','011','11',"But Jehoshabeath, the daughter of the king, took Joash the son of Ahaziah, and stole him from among the king's sons that were slain, and put him and his nurse in a bedchamber. So Jehoshabeath, the daughter of king Jehoram, the wife of Jehoiada the priest, (for she was the sister of Ahaziah,) hid him from Athaliah, so that she slew him not."],
['2Chronicles','2Chr','22','012','12',"And he was with them hid in the house of God six years: and Athaliah reigned over the land."]
]

SecondChronicles23 = [
    ['2Chronicles','2Chr','23','001','1',"And in the seventh year Jehoiada strengthened himself, and took the captains of hundreds, Azariah the son of Jeroham, and Ishmael the son of Jehohanan, and Azariah the son of Obed, and Maaseiah the son of Adaiah, and Elishaphat the son of Zichri, into covenant with him."],
['2Chronicles','2Chr','23','002','2',"And they went about in Judah, and gathered the Levites out of all the cities of Judah, and the chief of the fathers of Israel, and they came to Jerusalem."],
['2Chronicles','2Chr','23','003','3',"And all the congregation made a covenant with the king in the house of God. And he said unto them, Behold, the king's son shall reign, as the LORD hath said of the sons of David."],
['2Chronicles','2Chr','23','004','4',"This \emph{is} the thing that ye shall do; A third part of you entering on the sabbath, of the priests and of the Levites, \emph{shall} \emph{be} porters of the doors;"],
['2Chronicles','2Chr','23','005','5',"And a third part \emph{shall} \emph{be} at the king's house; and a third part at the gate of the foundation: and all the people \emph{shall} \emph{be} in the courts of the house of the LORD."],
['2Chronicles','2Chr','23','006','6',"But let none come into the house of the LORD, save the priests, and they that minister of the Levites; they shall go in, for they \emph{are} holy: but all the people shall keep the watch of the LORD."],
['2Chronicles','2Chr','23','007','7',"And the Levites shall compass the king round about, every man with his weapons in his hand; and whosoever \emph{else} cometh into the house, he shall be put to death: but be ye with the king when he cometh in, and when he goeth out."],
['2Chronicles','2Chr','23','008','8',"So the Levites and all Judah did according to all things that Jehoiada the priest had commanded, and took every man his men that were to come in on the sabbath, with them that were to go \emph{out} on the sabbath: for Jehoiada the priest dismissed not the courses."],
['2Chronicles','2Chr','23','009','9',"Moreover Jehoiada the priest delivered to the captains of hundreds spears, and bucklers, and shields, that \emph{had} \emph{been} king David's, which \emph{were} in the house of God."],
['2Chronicles','2Chr','23','010','10',"And he set all the people, every man having his weapon in his hand, from the right side of the temple to the left side of the temple, along by the altar and the temple, by the king round about."],
['2Chronicles','2Chr','23','011','11',"Then they brought out the king's son, and put upon him the crown, and \emph{gave} \emph{him} the testimony, and made him king. And Jehoiada and his sons anointed him, and said, God save the king."],
['2Chronicles','2Chr','23','012','12',"Now when Athaliah heard the noise of the people running and praising the king, she came to the people into the house of the LORD:"],
['2Chronicles','2Chr','23','013','13',"And she looked, and, behold, the king stood at his pillar at the entering in, and the princes and the trumpets by the king: and all the people of the land rejoiced, and sounded with trumpets, also the singers with instruments of musick, and such as taught to sing praise. Then Athaliah rent her clothes, and said, Treason, Treason."],
['2Chronicles','2Chr','23','014','14',"Then Jehoiada the priest brought out the captains of hundreds that were set over the host, and said unto them, Have her forth of the ranges: and whoso followeth her, let him be slain with the sword. For the priest said, Slay her not in the house of the LORD."],
['2Chronicles','2Chr','23','015','15',"So they laid hands on her; and when she was come to the entering of the horse gate by the king's house, they slew her there."],
['2Chronicles','2Chr','23','016','16',"And Jehoiada made a covenant between him, and between all the people, and between the king, that they should be the LORD'S people."],
['2Chronicles','2Chr','23','017','17',"Then all the people went to the house of Baal, and brake it down, and brake his altars and his images in pieces, and slew Mattan the priest of Baal before the altars."],
['2Chronicles','2Chr','23','018','18',"Also Jehoiada appointed the offices of the house of the LORD by the hand of the priests the Levites, whom David had distributed in the house of the LORD, to offer the burnt offerings of the LORD, as \emph{it} \emph{is} written in the law of Moses, with rejoicing and with singing, \emph{as} \emph{it} \emph{was} \emph{ordained} by David."],
['2Chronicles','2Chr','23','019','19',"And he set the porters at the gates of the house of the LORD, that none \emph{which} \emph{was} unclean in any thing should enter in."],
['2Chronicles','2Chr','23','020','20',"And he took the captains of hundreds, and the nobles, and the governors of the people, and all the people of the land, and brought down the king from the house of the LORD: and they came through the high gate into the king's house, and set the king upon the throne of the kingdom."],
['2Chronicles','2Chr','23','021','21',"And all the people of the land rejoiced: and the city was quiet, after that they had slain Athaliah with the sword."]
]

SecondChronicles24 = [
    ['2Chronicles','2Chr','24','001','1',"Joash \emph{was} seven years old when he began to reign, and he reigned forty years in Jerusalem. His mother's name also \emph{was} Zibiah of Beer-sheba."],
['2Chronicles','2Chr','24','002','2',"And Joash did \emph{that} \emph{which} \emph{was} right in the sight of the LORD all the days of Jehoiada the priest."],
['2Chronicles','2Chr','24','003','3',"And Jehoiada took for him two wives; and he begat sons and daughters."],
['2Chronicles','2Chr','24','004','4',"And it came to pass after this, \emph{that} Joash was minded to repair the house of the LORD."],
['2Chronicles','2Chr','24','005','5',"And he gathered together the priests and the Levites, and said to them, Go out unto the cities of Judah, and gather of all Israel money to repair the house of your God from year to year, and see that ye hasten the matter. Howbeit the Levites hastened \emph{it} not."],
['2Chronicles','2Chr','24','006','6',"And the king called for Jehoiada the chief, and said unto him, Why hast thou not required of the Levites to bring in out of Judah and out of Jerusalem the collection, \emph{according} \emph{to} \emph{the} \emph{commandment} of Moses the servant of the LORD, and of the congregation of Israel, for the tabernacle of witness?"],
['2Chronicles','2Chr','24','007','7',"For the sons of Athaliah, that wicked woman, had broken up the house of God; and also all the dedicated things of the house of the LORD did they bestow upon Baalim."],
['2Chronicles','2Chr','24','008','8',"And at the king's commandment they made a chest, and set it without at the gate of the house of the LORD."],
['2Chronicles','2Chr','24','009','9',"And they made a proclamation through Judah and Jerusalem, to bring in to the LORD the collection \emph{that} Moses the servant of God \emph{laid} upon Israel in the wilderness."],
['2Chronicles','2Chr','24','010','10',"And all the princes and all the people rejoiced, and brought in, and cast into the chest, until they had made an end."],
['2Chronicles','2Chr','24','011','11',"Now it came to pass, that at what time the chest was brought unto the king's office by the hand of the Levites, and when they saw that \emph{there} \emph{was} much money, the king's scribe and the high priest's officer came and emptied the chest, and took it, and carried it to his place again. Thus they did day by day, and gathered money in abundance."],
['2Chronicles','2Chr','24','012','12',"And the king and Jehoiada gave it to such as did the work of the service of the house of the LORD, and hired masons and carpenters to repair the house of the LORD, and also such as wrought iron and brass to mend the house of the LORD."],
['2Chronicles','2Chr','24','013','13',"So the workmen wrought, and the work was perfected by them, and they set the house of God in his state, and strengthened it."],
['2Chronicles','2Chr','24','014','14',"And when they had finished \emph{it}, they brought the rest of the money before the king and Jehoiada, whereof were made vessels for the house of the LORD, \emph{even} vessels to minister, and to offer \emph{withal}, and spoons, and vessels of gold and silver. And they offered burnt offerings in the house of the LORD continually all the days of Jehoiada."],
['2Chronicles','2Chr','24','015','15',"But Jehoiada waxed old, and was full of days when he died; an hundred and thirty years old \emph{was} \emph{he} when he died."],
['2Chronicles','2Chr','24','016','16',"And they buried him in the city of David among the kings, because he had done good in Israel, both toward God, and toward his house."],
['2Chronicles','2Chr','24','017','17',"Now after the death of Jehoiada came the princes of Judah, and made obeisance to the king. Then the king hearkened unto them."],
['2Chronicles','2Chr','24','018','18',"And they left the house of the LORD God of their fathers, and served groves and idols: and wrath came upon Judah and Jerusalem for this their trespass."],
['2Chronicles','2Chr','24','019','19',"Yet he sent prophets to them, to bring them again unto the LORD; and they testified against them: but they would not give ear."],
['2Chronicles','2Chr','24','020','20',"And the Spirit of God came upon Zechariah the son of Jehoiada the priest, which stood above the people, and said unto them, Thus saith God, Why transgress ye the commandments of the LORD, that ye cannot prosper? because ye have forsaken the LORD, he hath also forsaken you."],
['2Chronicles','2Chr','24','021','21',"And they conspired against him, and stoned him with stones at the commandment of the king in the court of the house of the LORD."],
['2Chronicles','2Chr','24','022','22',"Thus Joash the king remembered not the kindness which Jehoiada his father had done to him, but slew his son. And when he died, he said, The LORD look upon \emph{it}, and require \emph{it}."],
['2Chronicles','2Chr','24','023','23',"And it came to pass at the end of the year, \emph{that} the host of Syria came up against him: and they came to Judah and Jerusalem, and destroyed all the princes of the people from among the people, and sent all the spoil of them unto the king of Damascus."],
['2Chronicles','2Chr','24','024','24',"For the army of the Syrians came with a small company of men, and the LORD delivered a very great host into their hand, because they had forsaken the LORD God of their fathers. So they executed judgment against Joash."],
['2Chronicles','2Chr','24','025','25',"And when they were departed from him, (for they left him in great diseases,) his own servants conspired against him for the blood of the sons of Jehoiada the priest, and slew him on his bed, and he died: and they buried him in the city of David, but they buried him not in the sepulchres of the kings."],
['2Chronicles','2Chr','24','026','26',"And these are they that conspired against him; Zabad the son of Shimeath an Ammonitess, and Jehozabad the son of Shimrith a Moabitess."],
['2Chronicles','2Chr','24','027','27',"Now \emph{concerning} his sons, and the greatness of the burdens \emph{laid} upon him, and the repairing of the house of God, behold, they \emph{are} written in the story of the book of the kings. And Amaziah his son reigned in his stead."]
]

SecondChronicles25 = [
    ['2Chronicles','2Chr','25','001','1',"Amaziah \emph{was} twenty and five years old \emph{when} he began to reign, and he reigned twenty and nine years in Jerusalem. And his mother's name \emph{was} Jehoaddan of Jerusalem."],
['2Chronicles','2Chr','25','002','2',"And he did \emph{that} \emph{which} \emph{was} right in the sight of the LORD, but not with a perfect heart."],
['2Chronicles','2Chr','25','003','3',"Now it came to pass, when the kingdom was established to him, that he slew his servants that had killed the king his father."],
['2Chronicles','2Chr','25','004','4',"But he slew not their children, but \emph{did} as \emph{it} \emph{is} written in the law in the book of Moses, where the LORD commanded, saying, The fathers shall not die for the children, neither shall the children die for the fathers, but every man shall die for his own sin."],
['2Chronicles','2Chr','25','005','5',"Moreover Amaziah gathered Judah together, and made them captains over thousands, and captains over hundreds, according to the houses of \emph{their} fathers, throughout all Judah and Benjamin: and he numbered them from twenty years old and above, and found them three hundred thousand choice \emph{men,} \emph{able} to go forth to war, that could handle spear and shield."],
['2Chronicles','2Chr','25','006','6',"He hired also an hundred thousand mighty men of valour out of Israel for an hundred talents of silver."],
['2Chronicles','2Chr','25','007','7',"But there came a man of God to him, saying, O king, let not the army of Israel go with thee; for the LORD \emph{is} not with Israel, \emph{to} \emph{wit,} \emph{with} all the children of Ephraim."],
['2Chronicles','2Chr','25','008','8',"But if thou wilt go, do \emph{it}, be strong for the battle: God shall make thee fall before the enemy: for God hath power to help, and to cast down."],
['2Chronicles','2Chr','25','009','9',"And Amaziah said to the man of God, But what shall we do for the hundred talents which I have given to the army of Israel? And the man of God answered, The LORD is able to give thee much more than this."],
['2Chronicles','2Chr','25','010','10',"Then Amaziah separated them, \emph{to} \emph{wit}, the army that was come to him out of Ephraim, to go home again: wherefore their anger was greatly kindled against Judah, and they returned home in great anger."],
['2Chronicles','2Chr','25','011','11',"And Amaziah strengthened himself, and led forth his people, and went to the valley of salt, and smote of the children of Seir ten thousand."],
['2Chronicles','2Chr','25','012','12',"And \emph{other} ten thousand \emph{left} alive did the children of Judah carry away captive, and brought them unto the top of the rock, and cast them down from the top of the rock, that they all were broken in pieces."],
['2Chronicles','2Chr','25','013','13',"But the soldiers of the army which Amaziah sent back, that they should not go with him to battle, fell upon the cities of Judah, from Samaria even unto Beth-horon, and smote three thousand of them, and took much spoil."],
['2Chronicles','2Chr','25','014','14',"Now it came to pass, after that Amaziah was come from the slaughter of the Edomites, that he brought the gods of the children of Seir, and set them up \emph{to} \emph{be} his gods, and bowed down himself before them, and burned incense unto them."],
['2Chronicles','2Chr','25','015','15',"Wherefore the anger of the LORD was kindled against Amaziah, and he sent unto him a prophet, which said unto him, Why hast thou sought after the gods of the people, which could not deliver their own people out of thine hand?"],
['2Chronicles','2Chr','25','016','16',"And it came to pass, as he talked with him, that \emph{the} \emph{king} said unto him, Art thou made of the king's counsel? forbear; why shouldest thou be smitten? Then the prophet forbare, and said, I know that God hath determined to destroy thee, because thou hast done this, and hast not hearkened unto my counsel."],
['2Chronicles','2Chr','25','017','17',"Then Amaziah king of Judah took advice, and sent to Joash, the son of Jehoahaz, the son of Jehu, king of Israel, saying, Come, let us see one another in the face."],
['2Chronicles','2Chr','25','018','18',"And Joash king of Israel sent to Amaziah king of Judah, saying, The thistle that \emph{was} in Lebanon sent to the cedar that \emph{was} in Lebanon, saying, Give thy daughter to my son to wife: and there passed by a wild beast that \emph{was} in Lebanon, and trode down the thistle."],
['2Chronicles','2Chr','25','019','19',"Thou sayest, Lo, thou hast smitten the Edomites; and thine heart lifteth thee up to boast: abide now at home; why shouldest thou meddle to \emph{thine} hurt, that thou shouldest fall, \emph{even} thou, and Judah with thee?"],
['2Chronicles','2Chr','25','020','20',"But Amaziah would not hear; for it \emph{came} of God, that he might deliver them into the hand \emph{of} \emph{their} \emph{enemies}, because they sought after the gods of Edom."],
['2Chronicles','2Chr','25','021','21',"So Joash the king of Israel went up; and they saw one another in the face, \emph{both} he and Amaziah king of Judah, at Beth-shemesh, which \emph{belongeth} to Judah."],
['2Chronicles','2Chr','25','022','22',"And Judah was put to the worse before Israel, and they fled every man to his tent."],
['2Chronicles','2Chr','25','023','23',"And Joash the king of Israel took Amaziah king of Judah, the son of Joash, the son of Jehoahaz, at Beth-shemesh, and brought him to Jerusalem, and brake down the wall of Jerusalem from the gate of Ephraim to the corner gate, four hundred cubits."],
['2Chronicles','2Chr','25','024','24',"And \emph{he} \emph{took} all the gold and the silver, and all the vessels that were found in the house of God with Obed-edom, and the treasures of the king's house, the hostages also, and returned to Samaria."],
['2Chronicles','2Chr','25','025','25',"And Amaziah the son of Joash king of Judah lived after the death of Joash son of Jehoahaz king of Israel fifteen years."],
['2Chronicles','2Chr','25','026','26',"Now the rest of the acts of Amaziah, first and last, behold, \emph{are} they not written in the book of the kings of Judah and Israel?"],
['2Chronicles','2Chr','25','027','27',"Now after the time that Amaziah did turn away from following the LORD they made a conspiracy against him in Jerusalem; and he fled to Lachish: but they sent to Lachish after him, and slew him there."],
['2Chronicles','2Chr','25','028','28',"And they brought him upon horses, and buried him with his fathers in the city of Judah."]
]

SecondChronicles26 = [
    ['2Chronicles','2Chr','26','001','1',"Then all the people of Judah took Uzziah, who \emph{was} sixteen years old, and made him king in the room of his father Amaziah."],
['2Chronicles','2Chr','26','002','2',"He built Eloth, and restored it to Judah, after that the king slept with his fathers."],
['2Chronicles','2Chr','26','003','3',"Sixteen years old \emph{was} Uzziah when he began to reign, and he reigned fifty and two years in Jerusalem. His mother's name also \emph{was} Jecoliah of Jerusalem."],
['2Chronicles','2Chr','26','004','4',"And he did \emph{that} \emph{which} \emph{was} right in the sight of the LORD, according to all that his father Amaziah did."],
['2Chronicles','2Chr','26','005','5',"And he sought God in the days of Zechariah, who had understanding in the visions of God: and as long as he sought the LORD, God made him to prosper."],
['2Chronicles','2Chr','26','006','6',"And he went forth and warred against the Philistines, and brake down the wall of Gath, and the wall of Jabneh, and the wall of Ashdod, and built cities about Ashdod, and among the Philistines."],
['2Chronicles','2Chr','26','007','7',"And God helped him against the Philistines, and against the Arabians that dwelt in Gurbaal, and the Mehunims."],
['2Chronicles','2Chr','26','008','8',"And the Ammonites gave gifts to Uzziah: and his name spread abroad \emph{even} to the entering in of Egypt; for he strengthened \emph{himself} exceedingly."],
['2Chronicles','2Chr','26','009','9',"Moreover Uzziah built towers in Jerusalem at the corner gate, and at the valley gate, and at the turning \emph{of} \emph{the} \emph{wall}, and fortified them."],
['2Chronicles','2Chr','26','010','10',"Also he built towers in the desert, and digged many wells: for he had much cattle, both in the low country, and in the plains: husbandmen \emph{also}, and vine dressers in the mountains, and in Carmel: for he loved husbandry."],
['2Chronicles','2Chr','26','011','11',"Moreover Uzziah had an host of fighting men, that went out to war by bands, according to the number of their account by the hand of Jeiel the scribe and Maaseiah the ruler, under the hand of Hananiah, \emph{one} of the king's captains."],
['2Chronicles','2Chr','26','012','12',"The whole number of the chief of the fathers of the mighty men of valour \emph{were} two thousand and six hundred."],
['2Chronicles','2Chr','26','013','13',"And under their hand \emph{was} an army, three hundred thousand and seven thousand and five hundred, that made war with mighty power, to help the king against the enemy."],
['2Chronicles','2Chr','26','014','14',"And Uzziah prepared for them throughout all the host shields, and spears, and helmets, and habergeons, and bows, and slings \emph{to} \emph{cast} stones."],
['2Chronicles','2Chr','26','015','15',"And he made in Jerusalem engines, invented by cunning men, to be on the towers and upon the bulwarks, to shoot arrows and great stones withal. And his name spread far abroad; for he was marvellously helped, till he was strong."],
['2Chronicles','2Chr','26','016','16',"But when he was strong, his heart was lifted up to \emph{his} destruction: for he transgressed against the LORD his God, and went into the temple of the LORD to burn incense upon the altar of incense."],
['2Chronicles','2Chr','26','017','17',"And Azariah the priest went in after him, and with him fourscore priests of the LORD, \emph{that} \emph{were} valiant men:"],
['2Chronicles','2Chr','26','018','18',"And they withstood Uzziah the king, and said unto him, \emph{It} \emph{appertaineth} not unto thee, Uzziah, to burn incense unto the LORD, but to the priests the sons of Aaron, that are consecrated to burn incense: go out of the sanctuary; for thou hast trespassed; neither \emph{shall} \emph{it} \emph{be} for thine honour from the LORD God."],
['2Chronicles','2Chr','26','019','19',"Then Uzziah was wroth, and \emph{had} a censer in his hand to burn incense: and while he was wroth with the priests, the leprosy even rose up in his forehead before the priests in the house of the LORD, from beside the incense altar."],
['2Chronicles','2Chr','26','020','20',"And Azariah the chief priest, and all the priests, looked upon him, and, behold, he \emph{was} leprous in his forehead, and they thrust him out from thence; yea, himself hasted also to go out, because the LORD had smitten him."],
['2Chronicles','2Chr','26','021','21',"And Uzziah the king was a leper unto the day of his death, and dwelt in a several house, \emph{being} a leper; for he was cut off from the house of the LORD: and Jotham his son \emph{was} over the king's house, judging the people of the land."],
['2Chronicles','2Chr','26','022','22',"Now the rest of the acts of Uzziah, first and last, did Isaiah the prophet, the son of Amoz, write."],
['2Chronicles','2Chr','26','023','23',"So Uzziah slept with his fathers, and they buried him with his fathers in the field of the burial which \emph{belonged} to the kings; for they said, He \emph{is} a leper: and Jotham his son reigned in his stead."]
]

SecondChronicles27 = [
    ['2Chronicles','2Chr','27','001','1',"Jotham \emph{was} twenty and five years old when he began to reign, and he reigned sixteen years in Jerusalem. His mother's name also \emph{was} Jerushah, the daughter of Zadok."],
['2Chronicles','2Chr','27','002','2',"And he did \emph{that} \emph{which} \emph{was} right in the sight of the LORD, according to all that his father Uzziah did: howbeit he entered not into the temple of the LORD. And the people did yet corruptly."],
['2Chronicles','2Chr','27','003','3',"He built the high gate of the house of the LORD, and on the wall of Ophel he built much."],
['2Chronicles','2Chr','27','004','4',"Moreover he built cities in the mountains of Judah, and in the forests he built castles and towers."],
['2Chronicles','2Chr','27','005','5',"He fought also with the king of the Ammonites, and prevailed against them. And the children of Ammon gave him the same year an hundred talents of silver, and ten thousand measures of wheat, and ten thousand of barley. So much did the children of Ammon pay unto him, both the second year, and the third."],
['2Chronicles','2Chr','27','006','6',"So Jotham became mighty, because he prepared his ways before the LORD his God."],
['2Chronicles','2Chr','27','007','7',"Now the rest of the acts of Jotham, and all his wars, and his ways, lo, they \emph{are} written in the book of the kings of Israel and Judah."],
['2Chronicles','2Chr','27','008','8',"He was five and twenty years old when he began to reign, and reigned sixteen years in Jerusalem."],
['2Chronicles','2Chr','27','009','9',"And Jotham slept with his fathers, and they buried him in the city of David: and Ahaz his son reigned in his stead."]
]

SecondChronicles28 = [
    ['2Chronicles','2Chr','28','001','1',"Ahaz \emph{was} twenty years old when he began to reign, and he reigned sixteen years in Jerusalem: but he did not \emph{that} \emph{which} \emph{was} right in the sight of the LORD, like David his father:"],
['2Chronicles','2Chr','28','002','2',"For he walked in the ways of the kings of Israel, and made also molten images for Baalim."],
['2Chronicles','2Chr','28','003','3',"Moreover he burnt incense in the valley of the son of Hinnom, and burnt his children in the fire, after the abominations of the heathen whom the LORD had cast out before the children of Israel."],
['2Chronicles','2Chr','28','004','4',"He sacrificed also and burnt incense in the high places, and on the hills, and under every green tree."],
['2Chronicles','2Chr','28','005','5',"Wherefore the LORD his God delivered him into the hand of the king of Syria; and they smote him, and carried away a great multitude of them captives, and brought \emph{them} to Damascus. And he was also delivered into the hand of the king of Israel, who smote him with a great slaughter."],
['2Chronicles','2Chr','28','006','6',"For Pekah the son of Remaliah slew in Judah an hundred and twenty thousand in one day, \emph{which} \emph{were} all valiant men; because they had forsaken the LORD God of their fathers."],
['2Chronicles','2Chr','28','007','7',"And Zichri, a mighty man of Ephraim, slew Maaseiah the king's son, and Azrikam the governor of the house, and Elkanah \emph{that} \emph{was} next to the king."],
['2Chronicles','2Chr','28','008','8',"And the children of Israel carried away captive of their brethren two hundred thousand, women, sons, and daughters, and took also away much spoil from them, and brought the spoil to Samaria."],
['2Chronicles','2Chr','28','009','9',"But a prophet of the LORD was there, whose name \emph{was} Oded: and he went out before the host that came to Samaria, and said unto them, Behold, because the LORD God of your fathers was wroth with Judah, he hath delivered them into your hand, and ye have slain them in a rage \emph{that} reacheth up unto heaven."],
['2Chronicles','2Chr','28','010','10',"And now ye purpose to keep under the children of Judah and Jerusalem for bondmen and bondwomen unto you: \emph{but} \emph{are} \emph{there} not with you, even with you, sins against the LORD your God?"],
['2Chronicles','2Chr','28','011','11',"Now hear me therefore, and deliver the captives again, which ye have taken captive of your brethren: for the fierce wrath of the LORD \emph{is} upon you."],
['2Chronicles','2Chr','28','012','12',"Then certain of the heads of the children of Ephraim, Azariah the son of Johanan, Berechiah the son of Meshillemoth, and Jehizkiah the son of Shallum, and Amasa the son of Hadlai, stood up against them that came from the war,"],
['2Chronicles','2Chr','28','013','13',"And said unto them, Ye shall not bring in the captives hither: for whereas we have offended against the LORD \emph{already}, ye intend to add \emph{more} to our sins and to our trespass: for our trespass is great, and \emph{there} \emph{is} fierce wrath against Israel."],
['2Chronicles','2Chr','28','014','14',"So the armed men left the captives and the spoil before the princes and all the congregation."],
['2Chronicles','2Chr','28','015','15',"And the men which were expressed by name rose up, and took the captives, and with the spoil clothed all that were naked among them, and arrayed them, and shod them, and gave them to eat and to drink, and anointed them, and carried all the feeble of them upon asses, and brought them to Jericho, the city of palm trees, to their brethren: then they returned to Samaria."],
['2Chronicles','2Chr','28','016','16',"At that time did king Ahaz send unto the kings of Assyria to help him."],
['2Chronicles','2Chr','28','017','17',"For again the Edomites had come and smitten Judah, and carried away captives."],
['2Chronicles','2Chr','28','018','18',"The Philistines also had invaded the cities of the low country, and of the south of Judah, and had taken Beth-shemesh, and Ajalon, and Gederoth, and Shocho with the villages thereof, and Timnah with the villages thereof, Gimzo also and the villages thereof: and they dwelt there."],
['2Chronicles','2Chr','28','019','19',"For the LORD brought Judah low because of Ahaz king of Israel; for he made Judah naked, and transgressed sore against the LORD."],
['2Chronicles','2Chr','28','020','20',"And Tilgath-pilneser king of Assyria came unto him, and distressed him, but strengthened him not."],
['2Chronicles','2Chr','28','021','21',"For Ahaz took away a portion \emph{out} of the house of the LORD, and \emph{out} of the house of the king, and of the princes, and gave \emph{it} unto the king of Assyria: but he helped him not."],
['2Chronicles','2Chr','28','022','22',"And in the time of his distress did he trespass yet more against the LORD: this \emph{is} \emph{that} king Ahaz."],
['2Chronicles','2Chr','28','023','23',"For he sacrificed unto the gods of Damascus, which smote him: and he said, Because the gods of the kings of Syria help them, \emph{therefore} will I sacrifice to them, that they may help me. But they were the ruin of him, and of all Israel."],
['2Chronicles','2Chr','28','024','24',"And Ahaz gathered together the vessels of the house of God, and cut in pieces the vessels of the house of God, and shut up the doors of the house of the LORD, and he made him altars in every corner of Jerusalem."],
['2Chronicles','2Chr','28','025','25',"And in every several city of Judah he made high places to burn incense unto other gods, and provoked to anger the LORD God of his fathers."],
['2Chronicles','2Chr','28','026','26',"Now the rest of his acts and of all his ways, first and last, behold, they \emph{are} written in the book of the kings of Judah and Israel."],
['2Chronicles','2Chr','28','027','27',"And Ahaz slept with his fathers, and they buried him in the city, \emph{even} in Jerusalem: but they brought him not into the sepulchres of the kings of Israel: and Hezekiah his son reigned in his stead."]
]

SecondChronicles29 = [
    ['2Chronicles','2Chr','29','001','1',"Hezekiah began to reign \emph{when} \emph{he} \emph{was} five and twenty years old, and he reigned nine and twenty years in Jerusalem. And his mother's name \emph{was} Abijah, the daughter of Zechariah."],
['2Chronicles','2Chr','29','002','2',"And he did \emph{that} \emph{which} \emph{was} right in the sight of the LORD, according to all that David his father had done."],
['2Chronicles','2Chr','29','003','3',"He in the first year of his reign, in the first month, opened the doors of the house of the LORD, and repaired them."],
['2Chronicles','2Chr','29','004','4',"And he brought in the priests and the Levites, and gathered them together into the east street,"],
['2Chronicles','2Chr','29','005','5',"And said unto them, Hear me, ye Levites, sanctify now yourselves, and sanctify the house of the LORD God of your fathers, and carry forth the filthiness out of the holy \emph{place}."],
['2Chronicles','2Chr','29','006','6',"For our fathers have trespassed, and done \emph{that} \emph{which} \emph{was} evil in the eyes of the LORD our God, and have forsaken him, and have turned away their faces from the habitation of the LORD, and turned \emph{their} backs."],
['2Chronicles','2Chr','29','007','7',"Also they have shut up the doors of the porch, and put out the lamps, and have not burned incense nor offered burnt offerings in the holy \emph{place} unto the God of Israel."],
['2Chronicles','2Chr','29','008','8',"Wherefore the wrath of the LORD was upon Judah and Jerusalem, and he hath delivered them to trouble, to astonishment, and to hissing, as ye see with your eyes."],
['2Chronicles','2Chr','29','009','9',"For, lo, our fathers have fallen by the sword, and our sons and our daughters and our wives \emph{are} in captivity for this."],
['2Chronicles','2Chr','29','010','10',"Now \emph{it} \emph{is} in mine heart to make a covenant with the LORD God of Israel, that his fierce wrath may turn away from us."],
['2Chronicles','2Chr','29','011','11',"My sons, be not now negligent: for the LORD hath chosen you to stand before him, to serve him, and that ye should minister unto him, and burn incense."],
['2Chronicles','2Chr','29','012','12',"Then the Levites arose, Mahath the son of Amasai, and Joel the son of Azariah, of the sons of the Kohathites: and of the sons of Merari, Kish the son of Abdi, and Azariah the son of Jehalelel: and of the Gershonites; Joah the son of Zimmah, and Eden the son of Joah:"],
['2Chronicles','2Chr','29','013','13',"And of the sons of Elizaphan; Shimri, and Jeiel: and of the sons of Asaph; Zechariah, and Mattaniah:"],
['2Chronicles','2Chr','29','014','14',"And of the sons of Heman; Jehiel, and Shimei: and of the sons of Jeduthun; Shemaiah, and Uzziel."],
['2Chronicles','2Chr','29','015','15',"And they gathered their brethren, and sanctified themselves, and came, according to the commandment of the king, by the words of the LORD, to cleanse the house of the LORD."],
['2Chronicles','2Chr','29','016','16',"And the priests went into the inner part of the house of the LORD, to cleanse \emph{it}, and brought out all the uncleanness that they found in the temple of the LORD into the court of the house of the LORD. And the Levites took \emph{it}, to carry \emph{it} out abroad into the brook Kidron."],
['2Chronicles','2Chr','29','017','17',"Now they began on the first \emph{day} of the first month to sanctify, and on the eighth day of the month came they to the porch of the LORD: so they sanctified the house of the LORD in eight days; and in the sixteenth day of the first month they made an end."],
['2Chronicles','2Chr','29','018','18',"Then they went in to Hezekiah the king, and said, We have cleansed all the house of the LORD, and the altar of burnt offering, with all the vessels thereof, and the shewbread table, with all the vessels thereof."],
['2Chronicles','2Chr','29','019','19',"Moreover all the vessels, which king Ahaz in his reign did cast away in his transgression, have we prepared and sanctified, and, behold, they \emph{are} before the altar of the LORD."],
['2Chronicles','2Chr','29','020','20',"Then Hezekiah the king rose early, and gathered the rulers of the city, and went up to the house of the LORD."],
['2Chronicles','2Chr','29','021','21',"And they brought seven bullocks, and seven rams, and seven lambs, and seven he goats, for a sin offering for the kingdom, and for the sanctuary, and for Judah. And he commanded the priests the sons of Aaron to offer \emph{them} on the altar of the LORD."],
['2Chronicles','2Chr','29','022','22',"So they killed the bullocks, and the priests received the blood, and sprinkled \emph{it} on the altar: likewise, when they had killed the rams, they sprinkled the blood upon the altar: they killed also the lambs, and they sprinkled the blood upon the altar."],
['2Chronicles','2Chr','29','023','23',"And they brought forth the he goats \emph{for} the sin offering before the king and the congregation; and they laid their hands upon them:"],
['2Chronicles','2Chr','29','024','24',"And the priests killed them, and they made reconciliation with their blood upon the altar, to make an atonement for all Israel: for the king commanded \emph{that} the burnt offering and the sin offering \emph{should} \emph{be} \emph{made} for all Israel."],
['2Chronicles','2Chr','29','025','25',"And he set the Levites in the house of the LORD with cymbals, with psalteries, and with harps, according to the commandment of David, and of Gad the king's seer, and Nathan the prophet: for \emph{so} \emph{was} the commandment of the LORD by his prophets."],
['2Chronicles','2Chr','29','026','26',"And the Levites stood with the instruments of David, and the priests with the trumpets."],
['2Chronicles','2Chr','29','027','27',"And Hezekiah commanded to offer the burnt offering upon the altar. And when the burnt offering began, the song of the LORD began \emph{also} with the trumpets, and with the instruments \emph{ordained} by David king of Israel."],
['2Chronicles','2Chr','29','028','28',"And all the congregation worshipped, and the singers sang, and the trumpeters sounded: \emph{and} all \emph{this} \emph{continued} until the burnt offering was finished."],
['2Chronicles','2Chr','29','029','29',"And when they had made an end of offering, the king and all that were present with him bowed themselves, and worshipped."],
['2Chronicles','2Chr','29','030','30',"Moreover Hezekiah the king and the princes commanded the Levites to sing praise unto the LORD with the words of David, and of Asaph the seer. And they sang praises with gladness, and they bowed their heads and worshipped."],
['2Chronicles','2Chr','29','031','31',"Then Hezekiah answered and said, Now ye have consecrated yourselves unto the LORD, come near and bring sacrifices and thank offerings into the house of the LORD. And the congregation brought in sacrifices and thank offerings; and as many as were of a free heart burnt offerings."],
['2Chronicles','2Chr','29','032','32',"And the number of the burnt offerings, which the congregation brought, was threescore and ten bullocks, an hundred rams, \emph{and} two hundred lambs: all these \emph{were} for a burnt offering to the LORD."],
['2Chronicles','2Chr','29','033','33',"And the consecrated things \emph{were} six hundred oxen and three thousand sheep."],
['2Chronicles','2Chr','29','034','34',"But the priests were too few, so that they could not flay all the burnt offerings: wherefore their brethren the Levites did help them, till the work was ended, and until the \emph{other} priests had sanctified themselves: for the Levites \emph{were} more upright in heart to sanctify themselves than the priests."],
['2Chronicles','2Chr','29','035','35',"And also the burnt offerings \emph{were} in abundance, with the fat of the peace offerings, and the drink offerings for \emph{every} burnt offering. So the service of the house of the LORD was set in order."],
['2Chronicles','2Chr','29','036','36',"And Hezekiah rejoiced, and all the people, that God had prepared the people: for the thing was \emph{done} suddenly."]
]

SecondChronicles30 = [
    ['2Chronicles','2Chr','30','001','1',"And Hezekiah sent to all Israel and Judah, and wrote letters also to Ephraim and Manasseh, that they should come to the house of the LORD at Jerusalem, to keep the passover unto the LORD God of Israel."],
['2Chronicles','2Chr','30','002','2',"For the king had taken counsel, and his princes, and all the congregation in Jerusalem, to keep the passover in the second month."],
['2Chronicles','2Chr','30','003','3',"For they could not keep it at that time, because the priests had not sanctified themselves sufficiently, neither had the people gathered themselves together to Jerusalem."],
['2Chronicles','2Chr','30','004','4',"And the thing pleased the king and all the congregation."],
['2Chronicles','2Chr','30','005','5',"So they established a decree to make proclamation throughout all Israel, from Beer-sheba even to Dan, that they should come to keep the passover unto the LORD God of Israel at Jerusalem: for they had not done \emph{it} of a long \emph{time} \emph{in} \emph{such} \emph{sort} as it was written."],
['2Chronicles','2Chr','30','006','6',"So the posts went with the letters from the king and his princes throughout all Israel and Judah, and according to the commandment of the king, saying, Ye children of Israel, turn again unto the LORD God of Abraham, Isaac, and Israel, and he will return to the remnant of you, that are escaped out of the hand of the kings of Assyria."],
['2Chronicles','2Chr','30','007','7',"And be not ye like your fathers, and like your brethren, which trespassed against the LORD God of their fathers, \emph{who} therefore gave them up to desolation, as ye see."],
['2Chronicles','2Chr','30','008','8',"Now be ye not stiffnecked, as your fathers \emph{were,} \emph{but} yield yourselves unto the LORD, and enter into his sanctuary, which he hath sanctified for ever: and serve the LORD your God, that the fierceness of his wrath may turn away from you."],
['2Chronicles','2Chr','30','009','9',"For if ye turn again unto the LORD, your brethren and your children \emph{shall} \emph{find} compassion before them that lead them captive, so that they shall come again into this land: for the LORD your God \emph{is} gracious and merciful, and will not turn away \emph{his} face from you, if ye return unto him."],
['2Chronicles','2Chr','30','010','10',"So the posts passed from city to city through the country of Ephraim and Manasseh even unto Zebulun: but they laughed them to scorn, and mocked them."],
['2Chronicles','2Chr','30','011','11',"Nevertheless divers of Asher and Manasseh and of Zebulun humbled themselves, and came to Jerusalem."],
['2Chronicles','2Chr','30','012','12',"Also in Judah the hand of God was to give them one heart to do the commandment of the king and of the princes, by the word of the LORD."],
['2Chronicles','2Chr','30','013','13',"And there assembled at Jerusalem much people to keep the feast of unleavened bread in the second month, a very great congregation."],
['2Chronicles','2Chr','30','014','14',"And they arose and took away the altars that \emph{were} in Jerusalem, and all the altars for incense took they away, and cast \emph{them} into the brook Kidron."],
['2Chronicles','2Chr','30','015','15',"Then they killed the passover on the fourteenth \emph{day} of the second month: and the priests and the Levites were ashamed, and sanctified themselves, and brought in the burnt offerings into the house of the LORD."],
['2Chronicles','2Chr','30','016','16',"And they stood in their place after their manner, according to the law of Moses the man of God: the priests sprinkled the blood, \emph{which} \emph{they} \emph{received} of the hand of the Levites."],
['2Chronicles','2Chr','30','017','17',"For \emph{there} \emph{were} many in the congregation that were not sanctified: therefore the Levites had the charge of the killing of the passovers for every one \emph{that} \emph{was} not clean, to sanctify \emph{them} unto the LORD."],
['2Chronicles','2Chr','30','018','18',"For a multitude of the people, \emph{even} many of Ephraim, and Manasseh, Issachar, and Zebulun, had not cleansed themselves, yet did they eat the passover otherwise than it was written. But Hezekiah prayed for them, saying, The good LORD pardon every one"],
['2Chronicles','2Chr','30','019','19',"\emph{That} prepareth his heart to seek God, the LORD God of his fathers, though \emph{he} \emph{be} not \emph{cleansed} according to the purification of the sanctuary."],
['2Chronicles','2Chr','30','020','20',"And the LORD hearkened to Hezekiah, and healed the people."],
['2Chronicles','2Chr','30','021','21',"And the children of Israel that were present at Jerusalem kept the feast of unleavened bread seven days with great gladness: and the Levites and the priests praised the LORD day by day, \emph{singing} with loud instruments unto the LORD."],
['2Chronicles','2Chr','30','022','22',"And Hezekiah spake comfortably unto all the Levites that taught the good knowledge of the LORD: and they did eat throughout the feast seven days, offering peace offerings, and making confession to the LORD God of their fathers."],
['2Chronicles','2Chr','30','023','23',"And the whole assembly took counsel to keep other seven days: and they kept \emph{other} seven days with gladness."],
['2Chronicles','2Chr','30','024','24',"For Hezekiah king of Judah did give to the congregation a thousand bullocks and seven thousand sheep; and the princes gave to the congregation a thousand bullocks and ten thousand sheep: and a great number of priests sanctified themselves."],
['2Chronicles','2Chr','30','025','25',"And all the congregation of Judah, with the priests and the Levites, and all the congregation that came out of Israel, and the strangers that came out of the land of Israel, and that dwelt in Judah, rejoiced."],
['2Chronicles','2Chr','30','026','26',"So there was great joy in Jerusalem: for since the time of Solomon the son of David king of Israel \emph{there} \emph{was} not the like in Jerusalem."],
['2Chronicles','2Chr','30','027','27',"Then the priests the Levites arose and blessed the people: and their voice was heard, and their prayer came \emph{up} to his holy dwelling place, \emph{even} unto heaven."]
]

SecondChronicles31 = [
    ['2Chronicles','2Chr','31','001','1',"Now when all this was finished, all Israel that were present went out to the cities of Judah, and brake the images in pieces, and cut down the groves, and threw down the high places and the altars out of all Judah and Benjamin, in Ephraim also and Manasseh, until they had utterly destroyed them all. Then all the children of Israel returned, every man to his possession, into their own cities."],
['2Chronicles','2Chr','31','002','2',"And Hezekiah appointed the courses of the priests and the Levites after their courses, every man according to his service, the priests and Levites for burnt offerings and for peace offerings, to minister, and to give thanks, and to praise in the gates of the tents of the LORD."],
['2Chronicles','2Chr','31','003','3',"\emph{He} \emph{appointed} also the king's portion of his substance for the burnt offerings, \emph{to} \emph{wit}, for the morning and evening burnt offerings, and the burnt offerings for the sabbaths, and for the new moons, and for the set feasts, as \emph{it} \emph{is} written in the law of the LORD."],
['2Chronicles','2Chr','31','004','4',"Moreover he commanded the people that dwelt in Jerusalem to give the portion of the priests and the Levites, that they might be encouraged in the law of the LORD."],
['2Chronicles','2Chr','31','005','5',"And as soon as the commandment came abroad, the children of Israel brought in abundance the firstfruits of corn, wine, and oil, and honey, and of all the increase of the field; and the tithe of all \emph{things} brought they in abundantly."],
['2Chronicles','2Chr','31','006','6',"And \emph{concerning} the children of Israel and Judah, that dwelt in the cities of Judah, they also brought in the tithe of oxen and sheep, and the tithe of holy things which were consecrated unto the LORD their God, and laid \emph{them} by heaps."],
['2Chronicles','2Chr','31','007','7',"In the third month they began to lay the foundation of the heaps, and finished \emph{them} in the seventh month."],
['2Chronicles','2Chr','31','008','8',"And when Hezekiah and the princes came and saw the heaps, they blessed the LORD, and his people Israel."],
['2Chronicles','2Chr','31','009','9',"Then Hezekiah questioned with the priests and the Levites concerning the heaps."],
['2Chronicles','2Chr','31','010','10',"And Azariah the chief priest of the house of Zadok answered him, and said, Since \emph{the} \emph{people} began to bring the offerings into the house of the LORD, we have had enough to eat, and have left plenty: for the LORD hath blessed his people; and that which is left \emph{is} this great store."],
['2Chronicles','2Chr','31','011','11',"Then Hezekiah commanded to prepare chambers in the house of the LORD; and they prepared \emph{them},"],
['2Chronicles','2Chr','31','012','12',"And brought in the offerings and the tithes and the dedicated \emph{things} faithfully: over which Cononiah the Levite \emph{was} ruler, and Shimei his brother \emph{was} the next."],
['2Chronicles','2Chr','31','013','13',"And Jehiel, and Azaziah, and Nahath, and Asahel, and Jerimoth, and Jozabad, and Eliel, and Ismachiah, and Mahath, and Benaiah, \emph{were} overseers under the hand of Cononiah and Shimei his brother, at the commandment of Hezekiah the king, and Azariah the ruler of the house of God."],
['2Chronicles','2Chr','31','014','14',"And Kore the son of Imnah the Levite, the porter toward the east, \emph{was} over the freewill offerings of God, to distribute the oblations of the LORD, and the most holy things."],
['2Chronicles','2Chr','31','015','15',"And next him \emph{were} Eden, and Miniamin, and Jeshua, and Shemaiah, Amariah, and Shecaniah, in the cities of the priests, in \emph{their} set office, to give to their brethren by courses, as well to the great as to the small:"],
['2Chronicles','2Chr','31','016','16',"Beside their genealogy of males, from three years old and upward, \emph{even} unto every one that entereth into the house of the LORD, his daily portion for their service in their charges according to their courses;"],
['2Chronicles','2Chr','31','017','17',"Both to the genealogy of the priests by the house of their fathers, and the Levites from twenty years old and upward, in their charges by their courses;"],
['2Chronicles','2Chr','31','018','18',"And to the genealogy of all their little ones, their wives, and their sons, and their daughters, through all the congregation: for in their set office they sanctified themselves in holiness:"],
['2Chronicles','2Chr','31','019','19',"Also of the sons of Aaron the priests, \emph{which} \emph{were} in the fields of the suburbs of their cities, in every several city, the men that were expressed by name, to give portions to all the males among the priests, and to all that were reckoned by genealogies among the Levites."],
['2Chronicles','2Chr','31','020','20',"And thus did Hezekiah throughout all Judah, and wrought \emph{that} \emph{which} \emph{was} good and right and truth before the LORD his God."],
['2Chronicles','2Chr','31','021','21',"And in every work that he began in the service of the house of God, and in the law, and in the commandments, to seek his God, he did \emph{it} with all his heart, and prospered."]
]

SecondChronicles32 = [
    ['2Chronicles','2Chr','32','001','1',"After these things, and the establishment thereof, Sennacherib king of Assyria came, and entered into Judah, and encamped against the fenced cities, and thought to win them for himself."],
    ['2Chronicles','2Chr','32','002','2',"And when Hezekiah saw that Sennacherib was come, and that he was purposed to fight against Jerusalem,"],
['2Chronicles','2Chr','32','003','3',"He took counsel with his princes and his mighty men to stop the waters of the fountains which \emph{were} without the city: and they did help him."],
['2Chronicles','2Chr','32','004','4',"So there was gathered much people together, who stopped all the fountains, and the brook that ran through the midst of the land, saying, Why should the kings of Assyria come, and find much water?"],
['2Chronicles','2Chr','32','005','5',"Also he strengthened himself, and built up all the wall that was broken, and raised \emph{it} up to the towers, and another wall without, and repaired Millo \emph{in} the city of David, and made darts and shields in abundance."],
['2Chronicles','2Chr','32','006','6',"And he set captains of war over the people, and gathered them together to him in the street of the gate of the city, and spake comfortably to them, saying,"],
['2Chronicles','2Chr','32','007','7',"Be strong and courageous, be not afraid nor dismayed for the king of Assyria, nor for all the multitude that \emph{is} with him: for \emph{there} \emph{be} more with us than with him:"],
['2Chronicles','2Chr','32','008','8',"With him \emph{is} an arm of flesh; but with us \emph{is} the LORD our God to help us, and to fight our battles. And the people rested themselves upon the words of Hezekiah king of Judah."],
['2Chronicles','2Chr','32','009','9',"After this did Sennacherib king of Assyria send his servants to Jerusalem, (but he \emph{himself} \emph{laid} \emph{siege} against Lachish, and all his power with him,) unto Hezekiah king of Judah, and unto all Judah that \emph{were} at Jerusalem, saying,"],
['2Chronicles','2Chr','32','010','10',"Thus saith Sennacherib king of Assyria, Whereon do ye trust, that ye abide in the siege in Jerusalem?"],
['2Chronicles','2Chr','32','011','11',"Doth not Hezekiah persuade you to give over yourselves to die by famine and by thirst, saying, The LORD our God shall deliver us out of the hand of the king of Assyria?"],
['2Chronicles','2Chr','32','012','12',"Hath not the same Hezekiah taken away his high places and his altars, and commanded Judah and Jerusalem, saying, Ye shall worship before one altar, and burn incense upon it?"],
['2Chronicles','2Chr','32','013','13',"Know ye not what I and my fathers have done unto all the people of \emph{other} lands? were the gods of the nations of those lands any ways able to deliver their lands out of mine hand?"],
['2Chronicles','2Chr','32','014','14',"Who \emph{was} \emph{there} among all the gods of those nations that my fathers utterly destroyed, that could deliver his people out of mine hand, that your God should be able to deliver you out of mine hand?"],
['2Chronicles','2Chr','32','015','15',"Now therefore let not Hezekiah deceive you, nor persuade you on this manner, neither yet believe him: for no god of any nation or kingdom was able to deliver his people out of mine hand, and out of the hand of my fathers: how much less shall your God deliver you out of mine hand?"],
['2Chronicles','2Chr','32','016','16',"And his servants spake yet \emph{more} against the LORD God, and against his servant Hezekiah."],
['2Chronicles','2Chr','32','017','17',"He wrote also letters to rail on the LORD God of Israel, and to speak against him, saying, As the gods of the nations of \emph{other} lands have not delivered their people out of mine hand, so shall not the God of Hezekiah deliver his people out of mine hand."],
['2Chronicles','2Chr','32','018','18',"Then they cried with a loud voice in the Jews' speech unto the people of Jerusalem that \emph{were} on the wall, to affright them, and to trouble them; that they might take the city."],
['2Chronicles','2Chr','32','019','19',"And they spake against the God of Jerusalem, as against the gods of the people of the earth, \emph{which} \emph{were} the work of the hands of man."],
['2Chronicles','2Chr','32','020','20',"And for this \emph{cause} Hezekiah the king, and the prophet Isaiah the son of Amoz, prayed and cried to heaven."],
['2Chronicles','2Chr','32','021','21',"And the LORD sent an angel, which cut off all the mighty men of valour, and the leaders and captains in the camp of the king of Assyria. So he returned with shame of face to his own land. And when he was come into the house of his god, they that came forth of his own bowels slew him there with the sword."],
['2Chronicles','2Chr','32','022','22',"Thus the LORD saved Hezekiah and the inhabitants of Jerusalem from the hand of Sennacherib the king of Assyria, and from the hand of all \emph{other}, and guided them on every side."],
['2Chronicles','2Chr','32','023','23',"And many brought gifts unto the LORD to Jerusalem, and presents to Hezekiah king of Judah: so that he was magnified in the sight of all nations from thenceforth."],
['2Chronicles','2Chr','32','024','24',"In those days Hezekiah was sick to the death, and prayed unto the LORD: and he spake unto him, and he gave him a sign."],
['2Chronicles','2Chr','32','025','25',"But Hezekiah rendered not again according to the benefit \emph{done} unto him; for his heart was lifted up: therefore there was wrath upon him, and upon Judah and Jerusalem."],
['2Chronicles','2Chr','32','026','26',"Notwithstanding Hezekiah humbled himself for the pride of his heart, \emph{both} he and the inhabitants of Jerusalem, so that the wrath of the LORD came not upon them in the days of Hezekiah."],
['2Chronicles','2Chr','32','027','27',"And Hezekiah had exceeding much riches and honour: and he made himself treasuries for silver, and for gold, and for precious stones, and for spices, and for shields, and for all manner of pleasant jewels;"],
['2Chronicles','2Chr','32','028','28',"Storehouses also for the increase of corn, and wine, and oil; and stalls for all manner of beasts, and cotes for flocks."],
['2Chronicles','2Chr','32','029','29',"Moreover he provided him cities, and possessions of flocks and herds in abundance: for God had given him substance very much."],
['2Chronicles','2Chr','32','030','30',"This same Hezekiah also stopped the upper watercourse of Gihon, and brought it straight down to the west side of the city of David. And Hezekiah prospered in all his works."],
['2Chronicles','2Chr','32','031','31',"Howbeit in \emph{the} \emph{business} \emph{of} the ambassadors of the princes of Babylon, who sent unto him to enquire of the wonder that was \emph{done} in the land, God left him, to try him, that he might know all \emph{that} \emph{was} in his heart."],
    ['2Chronicles','2Chr','32','032','32',"Now the rest of the acts of Hezekiah, and his goodness, behold, they \emph{are} written in the vision of Isaiah the prophet, the son of Amoz, \emph{and} in the book of the kings of Judah and Israel."],
    ['2Chronicles','2Chr','32','033','33',"And Hezekiah slept with his fathers, and they buried him in the chiefest of the sepulchres of the sons of David: and all Judah and the inhabitants of Jerusalem did him honour at his death. And Manasseh his son reigned in his stead."]
]

SecondChronicles33 = [
    ['2Chronicles','2Chr','33','001','1',"Manasseh \emph{was} twelve years old when he began to reign, and he reigned fifty and five years in Jerusalem:"],
    ['2Chronicles','2Chr','33','002','2',"But did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD, like unto the abominations of the heathen, whom the LORD had cast out before the children of Israel."],
    ['2Chronicles','2Chr','33','003','3',"For he built again the high places which Hezekiah his father had broken down, and he reared up altars for Baalim, and made groves, and worshipped all the host of heaven, and served them."],
    ['2Chronicles','2Chr','33','004','4',"Also he built altars in the house of the LORD, whereof the LORD had said, In Jerusalem shall my name be for ever."],
    ['2Chronicles','2Chr','33','005','5',"And he built altars for all the host of heaven in the two courts of the house of the LORD."],
    ['2Chronicles','2Chr','33','006','6',"And he caused his children to pass through the fire in the valley of the son of Hinnom: also he observed times, and used enchantments, and used witchcraft, and dealt with a familiar spirit, and with wizards: he wrought much evil in the sight of the LORD, to provoke him to anger."],
    ['2Chronicles','2Chr','33','007','7',"And he set a carved image, the idol which he had made, in the house of God, of which God had said to David and to Solomon his son, In this house, and in Jerusalem, which I have chosen before all the tribes of Israel, will I put my name for ever:"],
    ['2Chronicles','2Chr','33','008','8',"Neither will I any more remove the foot of Israel from out of the land which I have appointed for your fathers; so that they will take heed to do all that I have commanded them, according to the whole law and the statutes and the ordinances by the hand of Moses."],
    ['2Chronicles','2Chr','33','009','9',"So Manasseh made Judah and the inhabitants of Jerusalem to err, \emph{and} to do worse than the heathen, whom the LORD had destroyed before the children of Israel."],
    ['2Chronicles','2Chr','33','010','10',"And the LORD spake to Manasseh, and to his people: but they would not hearken."],
    ['2Chronicles','2Chr','33','011','11',"Wherefore the LORD brought upon them the captains of the host of the king of Assyria, which took Manasseh among the thorns, and bound him with fetters, and carried him to Babylon."],
    ['2Chronicles','2Chr','33','012','12',"And when he was in affliction, he besought the LORD his God, and humbled himself greatly before the God of his fathers,"],
    ['2Chronicles','2Chr','33','013','13',"And prayed unto him: and he was intreated of him, and heard his supplication, and brought him again to Jerusalem into his kingdom. Then Manasseh knew that the LORD he \emph{was} God."],
    ['2Chronicles','2Chr','33','014','14',"Now after this he built a wall without the city of David, on the west side of Gihon, in the valley, even to the entering in at the fish gate, and compassed about Ophel, and raised it up a very great height, and put captains of war in all the fenced cities of Judah."],
    ['2Chronicles','2Chr','33','015','15',"And he took away the strange gods, and the idol out of the house of the LORD, and all the altars that he had built in the mount of the house of the LORD, and in Jerusalem, and cast \emph{them} out of the city."],
    ['2Chronicles','2Chr','33','016','16',"And he repaired the altar of the LORD, and sacrificed thereon peace offerings and thank offerings, and commanded Judah to serve the LORD God of Israel."],
    ['2Chronicles','2Chr','33','017','17',"Nevertheless the people did sacrifice still in the high places, \emph{yet} unto the LORD their God only."],
    ['2Chronicles','2Chr','33','018','18',"Now the rest of the acts of Manasseh, and his prayer unto his God, and the words of the seers that spake to him in the name of the LORD God of Israel, behold, they \emph{are} \emph{written} in the book of the kings of Israel."],
    ['2Chronicles','2Chr','33','019','19',"His prayer also, and \emph{how} \emph{God} was intreated of him, and all his sin, and his trespass, and the places wherein he built high places, and set up groves and graven images, before he was humbled: behold, they \emph{are} written among the sayings of the seers."],
    ['2Chronicles','2Chr','33','020','20',"So Manasseh slept with his fathers, and they buried him in his own house: and Amon his son reigned in his stead."],
    ['2Chronicles','2Chr','33','021','21',"Amon \emph{was} two and twenty years old when he began to reign, and reigned two years in Jerusalem."],
    ['2Chronicles','2Chr','33','022','22',"But he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD, as did Manasseh his father: for Amon sacrificed unto all the carved images which Manasseh his father had made, and served them;"],
    ['2Chronicles','2Chr','33','023','23',"And humbled not himself before the LORD, as Manasseh his father had humbled himself; but Amon trespassed more and more."],
    ['2Chronicles','2Chr','33','024','24',"And his servants conspired against him, and slew him in his own house."],
    ['2Chronicles','2Chr','33','025','25',"But the people of the land slew all them that had conspired against king Amon; and the people of the land made Josiah his son king in his stead."]
]

SecondChronicles34 = [
    ['2Chronicles','2Chr','34','001','1',"Josiah \emph{was} eight years old when he began to reign, and he reigned in Jerusalem one and thirty years."],
['2Chronicles','2Chr','34','002','2',"And he did \emph{that} \emph{which} \emph{was} right in the sight of the LORD, and walked in the ways of David his father, and declined \emph{neither} to the right hand, nor to the left."],
['2Chronicles','2Chr','34','003','3',"For in the eighth year of his reign, while he was yet young, he began to seek after the God of David his father: and in the twelfth year he began to purge Judah and Jerusalem from the high places, and the groves, and the carved images, and the molten images."],
['2Chronicles','2Chr','34','004','4',"And they brake down the altars of Baalim in his presence; and the images, that \emph{were} on high above them, he cut down; and the groves, and the carved images, and the molten images, he brake in pieces, and made dust \emph{of} \emph{them}, and strowed \emph{it} upon the graves of them that had sacrificed unto them."],
['2Chronicles','2Chr','34','005','5',"And he burnt the bones of the priests upon their altars, and cleansed Judah and Jerusalem."],
['2Chronicles','2Chr','34','006','6',"And \emph{so} \emph{did} \emph{he} in the cities of Manasseh, and Ephraim, and Simeon, even unto Naphtali, with their mattocks round about."],
['2Chronicles','2Chr','34','007','7',"And when he had broken down the altars and the groves, and had beaten the graven images into powder, and cut down all the idols throughout all the land of Israel, he returned to Jerusalem."],
['2Chronicles','2Chr','34','008','8',"Now in the eighteenth year of his reign, when he had purged the land, and the house, he sent Shaphan the son of Azaliah, and Maaseiah the governor of the city, and Joah the son of Joahaz the recorder, to repair the house of the LORD his God."],
['2Chronicles','2Chr','34','009','9',"And when they came to Hilkiah the high priest, they delivered the money that was brought into the house of God, which the Levites that kept the doors had gathered of the hand of Manasseh and Ephraim, and of all the remnant of Israel, and of all Judah and Benjamin; and they returned to Jerusalem."],
['2Chronicles','2Chr','34','010','10',"And they put \emph{it} in the hand of the workmen that had the oversight of the house of the LORD, and they gave it to the workmen that wrought in the house of the LORD, to repair and amend the house:"],
['2Chronicles','2Chr','34','011','11',"Even to the artificers and builders gave they \emph{it}, to buy hewn stone, and timber for couplings, and to floor the houses which the kings of Judah had destroyed."],
['2Chronicles','2Chr','34','012','12',"And the men did the work faithfully: and the overseers of them \emph{were} Jahath and Obadiah, the Levites, of the sons of Merari; and Zechariah and Meshullam, of the sons of the Kohathites, to set \emph{it} forward; and \emph{other} \emph{of} the Levites, all that could skill of instruments of musick."],
['2Chronicles','2Chr','34','013','13',"Also \emph{they} \emph{were} over the bearers of burdens, and \emph{were} overseers of all that wrought the work in any manner of service: and of the Levites \emph{there} \emph{were} scribes, and officers, and porters."],
['2Chronicles','2Chr','34','014','14',"And when they brought out the money that was brought into the house of the LORD, Hilkiah the priest found a book of the law of the LORD \emph{given} by Moses."],
['2Chronicles','2Chr','34','015','15',"And Hilkiah answered and said to Shaphan the scribe, I have found the book of the law in the house of the LORD. And Hilkiah delivered the book to Shaphan."],
['2Chronicles','2Chr','34','016','16',"And Shaphan carried the book to the king, and brought the king word back again, saying, All that was committed to thy servants, they do \emph{it}."],
['2Chronicles','2Chr','34','017','17',"And they have gathered together the money that was found in the house of the LORD, and have delivered it into the hand of the overseers, and to the hand of the workmen."],
['2Chronicles','2Chr','34','018','18',"Then Shaphan the scribe told the king, saying, Hilkiah the priest hath given me a book. And Shaphan read it before the king."],
['2Chronicles','2Chr','34','019','19',"And it came to pass, when the king had heard the words of the law, that he rent his clothes."],
['2Chronicles','2Chr','34','020','20',"And the king commanded Hilkiah, and Ahikam the son of Shaphan, and Abdon the son of Micah, and Shaphan the scribe, and Asaiah a servant of the king's, saying,"],
['2Chronicles','2Chr','34','021','21',"Go, enquire of the LORD for me, and for them that are left in Israel and in Judah, concerning the words of the book that is found: for great \emph{is} the wrath of the LORD that is poured out upon us, because our fathers have not kept the word of the LORD, to do after all that is written in this book."],
['2Chronicles','2Chr','34','022','22',"And Hilkiah, and \emph{they} that the king \emph{had} \emph{appointed}, went to Huldah the prophetess, the wife of Shallum the son of Tikvath, the son of Hasrah, keeper of the wardrobe; (now she dwelt in Jerusalem in the college:) and they spake to her to that \emph{effect}."],
['2Chronicles','2Chr','34','023','23',"And she answered them, Thus saith the LORD God of Israel, Tell ye the man that sent you to me,"],
['2Chronicles','2Chr','34','024','24',"Thus saith the LORD, Behold, I will bring evil upon this place, and upon the inhabitants thereof, \emph{even} all the curses that are written in the book which they have read before the king of Judah:"],
['2Chronicles','2Chr','34','025','25',"Because they have forsaken me, and have burned incense unto other gods, that they might provoke me to anger with all the works of their hands; therefore my wrath shall be poured out upon this place, and shall not be quenched."],
['2Chronicles','2Chr','34','026','26',"And as for the king of Judah, who sent you to enquire of the LORD, so shall ye say unto him, Thus saith the LORD God of Israel \emph{concerning} the words which thou hast heard;"],
['2Chronicles','2Chr','34','027','27',"Because thine heart was tender, and thou didst humble thyself before God, when thou heardest his words against this place, and against the inhabitants thereof, and humbledst thyself before me, and didst rend thy clothes, and weep before me; I have even heard \emph{thee} also, saith the LORD."],
['2Chronicles','2Chr','34','028','28',"Behold, I will gather thee to thy fathers, and thou shalt be gathered to thy grave in peace, neither shall thine eyes see all the evil that I will bring upon this place, and upon the inhabitants of the same. So they brought the king word again."],
['2Chronicles','2Chr','34','029','29',"Then the king sent and gathered together all the elders of Judah and Jerusalem."],
['2Chronicles','2Chr','34','030','30',"And the king went up into the house of the LORD, and all the men of Judah, and the inhabitants of Jerusalem, and the priests, and the Levites, and all the people, great and small: and he read in their ears all the words of the book of the covenant that was found in the house of the LORD."],
['2Chronicles','2Chr','34','031','31',"And the king stood in his place, and made a covenant before the LORD, to walk after the LORD, and to keep his commandments, and his testimonies, and his statutes, with all his heart, and with all his soul, to perform the words of the covenant which are written in this book."],
['2Chronicles','2Chr','34','032','32',"And he caused all that were present in Jerusalem and Benjamin to stand \emph{to} \emph{it}. And the inhabitants of Jerusalem did according to the covenant of God, the God of their fathers."],
['2Chronicles','2Chr','34','033','33',"And Josiah took away all the abominations out of all the countries that \emph{pertained} to the children of Israel, and made all that were present in Israel to serve, \emph{even} to serve the LORD their God. \emph{And} all his days they departed not from following the LORD, the God of their fathers."]
]

SecondChronicles35 = [
    ['2Chronicles','2Chr','35','001','1',"Moreover Josiah kept a passover unto the LORD in Jerusalem: and they killed the passover on the fourteenth \emph{day} of the first month."],
['2Chronicles','2Chr','35','002','2',"And he set the priests in their charges, and encouraged them to the service of the house of the LORD,"],
['2Chronicles','2Chr','35','003','3',"And said unto the Levites that taught all Israel, which were holy unto the LORD, Put the holy ark in the house which Solomon the son of David king of Israel did build; \emph{it} \emph{shall} not \emph{be} a burden upon \emph{your} shoulders: serve now the LORD your God, and his people Israel,"],
['2Chronicles','2Chr','35','004','4',"And prepare \emph{yourselves} by the houses of your fathers, after your courses, according to the writing of David king of Israel, and according to the writing of Solomon his son."],
['2Chronicles','2Chr','35','005','5',"And stand in the holy \emph{place} according to the divisions of the families of the fathers of your brethren the people, and \emph{after} the division of the families of the Levites."],
['2Chronicles','2Chr','35','006','6',"So kill the passover, and sanctify yourselves, and prepare your brethren, that \emph{they} may do according to the word of the LORD by the hand of Moses."],
['2Chronicles','2Chr','35','007','7',"And Josiah gave to the people, of the flock, lambs and kids, all for the passover offerings, for all that were present, to the number of thirty thousand, and three thousand bullocks: these \emph{were} of the king's substance."],
['2Chronicles','2Chr','35','008','8',"And his princes gave willingly unto the people, to the priests, and to the Levites: Hilkiah and Zechariah and Jehiel, rulers of the house of God, gave unto the priests for the passover offerings two thousand and six hundred \emph{small} \emph{cattle}, and three hundred oxen."],
['2Chronicles','2Chr','35','009','9',"Conaniah also, and Shemaiah and Nethaneel, his brethren, and Hashabiah and Jeiel and Jozabad, chief of the Levites, gave unto the Levites for passover offerings five thousand \emph{small} \emph{cattle}, and five hundred oxen."],
['2Chronicles','2Chr','35','010','10',"So the service was prepared, and the priests stood in their place, and the Levites in their courses, according to the king's commandment."],
['2Chronicles','2Chr','35','011','11',"And they killed the passover, and the priests sprinkled \emph{the} \emph{blood} from their hands, and the Levites flayed \emph{them}."],
['2Chronicles','2Chr','35','012','12',"And they removed the burnt offerings, that they might give according to the divisions of the families of the people, to offer unto the LORD, as \emph{it} \emph{is} written in the book of Moses. And so \emph{did} \emph{they} with the oxen."],
['2Chronicles','2Chr','35','013','13',"And they roasted the passover with fire according to the ordinance: but the \emph{other} holy \emph{offerings} sod they in pots, and in caldrons, and in pans, and divided \emph{them} speedily among all the people."],
['2Chronicles','2Chr','35','014','14',"And afterward they made ready for themselves, and for the priests: because the priests the sons of Aaron \emph{were} \emph{busied} in offering of burnt offerings and the fat until night; therefore the Levites prepared for themselves, and for the priests the sons of Aaron."],
['2Chronicles','2Chr','35','015','15',"And the singers the sons of Asaph \emph{were} in their place, according to the commandment of David, and Asaph, and Heman, and Jeduthun the king's seer; and the porters \emph{waited} at every gate; they might not depart from their service; for their brethren the Levites prepared for them."],
['2Chronicles','2Chr','35','016','16',"So all the service of the LORD was prepared the same day, to keep the passover, and to offer burnt offerings upon the altar of the LORD, according to the commandment of king Josiah."],
['2Chronicles','2Chr','35','017','17',"And the children of Israel that were present kept the passover at that time, and the feast of unleavened bread seven days."],
['2Chronicles','2Chr','35','018','18',"And there was no passover like to that kept in Israel from the days of Samuel the prophet; neither did all the kings of Israel keep such a passover as Josiah kept, and the priests, and the Levites, and all Judah and Israel that were present, and the inhabitants of Jerusalem."],
['2Chronicles','2Chr','35','019','19',"In the eighteenth year of the reign of Josiah was this passover kept."],
['2Chronicles','2Chr','35','020','20',"After all this, when Josiah had prepared the temple, Necho king of Egypt came up to fight against Carchemish by Euphrates: and Josiah went out against him."],
['2Chronicles','2Chr','35','021','21',"But he sent ambassadors to him, saying, What have I to do with thee, thou king of Judah? \emph{I} \emph{come} not against thee this day, but against the house wherewith I have war: for God commanded me to make haste: forbear thee from \emph{meddling} \emph{with} God, who \emph{is} with me, that he destroy thee not."],
['2Chronicles','2Chr','35','022','22',"Nevertheless Josiah would not turn his face from him, but disguised himself, that he might fight with him, and hearkened not unto the words of Necho from the mouth of God, and came to fight in the valley of Megiddo."],
['2Chronicles','2Chr','35','023','23',"And the archers shot at king Josiah; and the king said to his servants, Have me away; for I am sore wounded."],
['2Chronicles','2Chr','35','024','24',"His servants therefore took him out of that chariot, and put him in the second chariot that he had; and they brought him to Jerusalem, and he died, and was buried in \emph{one} \emph{of} the sepulchres of his fathers. And all Judah and Jerusalem mourned for Josiah."],
['2Chronicles','2Chr','35','025','25',"And Jeremiah lamented for Josiah: and all the singing men and the singing women spake of Josiah in their lamentations to this day, and made them an ordinance in Israel: and, behold, they \emph{are} written in the lamentations."],
['2Chronicles','2Chr','35','026','26',"Now the rest of the acts of Josiah, and his goodness, according to \emph{that} \emph{which} \emph{was} written in the law of the LORD,"],
['2Chronicles','2Chr','35','027','27',"And his deeds, first and last, behold, they \emph{are} written in the book of the kings of Israel and Judah."]
]

SecondChronicles36 = [
    ['2Chronicles','2Chr','36','001','1',"Then the people of the land took Jehoahaz the son of Josiah, and made him king in his father's stead in Jerusalem."],
['2Chronicles','2Chr','36','002','2',"Jehoahaz \emph{was} twenty and three years old when he began to reign, and he reigned three months in Jerusalem."],
['2Chronicles','2Chr','36','003','3',"And the king of Egypt put him down at Jerusalem, and condemned the land in an hundred talents of silver and a talent of gold."],
['2Chronicles','2Chr','36','004','4',"And the king of Egypt made Eliakim his brother king over Judah and Jerusalem, and turned his name to Jehoiakim. And Necho took Jehoahaz his brother, and carried him to Egypt."],
['2Chronicles','2Chr','36','005','5',"Jehoiakim \emph{was} twenty and five years old when he began to reign, and he reigned eleven years in Jerusalem: and he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD his God."],
['2Chronicles','2Chr','36','006','6',"Against him came up Nebuchadnezzar king of Babylon, and bound him in fetters, to carry him to Babylon."],
['2Chronicles','2Chr','36','007','7',"Nebuchadnezzar also carried of the vessels of the house of the LORD to Babylon, and put them in his temple at Babylon."],
['2Chronicles','2Chr','36','008','8',"Now the rest of the acts of Jehoiakim, and his abominations which he did, and that which was found in him, behold, they \emph{are} written in the book of the kings of Israel and Judah: and Jehoiachin his son reigned in his stead."],
['2Chronicles','2Chr','36','009','9',"Jehoiachin \emph{was} eight years old when he began to reign, and he reigned three months and ten days in Jerusalem: and he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD."],
['2Chronicles','2Chr','36','010','10',"And when the year was expired, king Nebuchadnezzar sent, and brought him to Babylon, with the goodly vessels of the house of the LORD, and made Zedekiah his brother king over Judah and Jerusalem."],
['2Chronicles','2Chr','36','011','11',"Zedekiah \emph{was} one and twenty years old when he began to reign, and reigned eleven years in Jerusalem."],
['2Chronicles','2Chr','36','012','12',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD his God, \emph{and} humbled not himself before Jeremiah the prophet \emph{speaking} from the mouth of the LORD."],
['2Chronicles','2Chr','36','013','13',"And he also rebelled against king Nebuchadnezzar, who had made him swear by God: but he stiffened his neck, and hardened his heart from turning unto the LORD God of Israel."],
['2Chronicles','2Chr','36','014','14',"Moreover all the chief of the priests, and the people, transgressed very much after all the abominations of the heathen; and polluted the house of the LORD which he had hallowed in Jerusalem."],
['2Chronicles','2Chr','36','015','15',"And the LORD God of their fathers sent to them by his messengers, rising up betimes, and sending; because he had compassion on his people, and on his dwelling place:"],
['2Chronicles','2Chr','36','016','16',"But they mocked the messengers of God, and despised his words, and misused his prophets, until the wrath of the LORD arose against his people, till \emph{there} \emph{was} no remedy."],
['2Chronicles','2Chr','36','017','17',"Therefore he brought upon them the king of the Chaldees, who slew their young men with the sword in the house of their sanctuary, and had no compassion upon young man or maiden, old man, or him that stooped for age: he gave \emph{them} all into his hand."],
['2Chronicles','2Chr','36','018','18',"And all the vessels of the house of God, great and small, and the treasures of the house of the LORD, and the treasures of the king, and of his princes; all \emph{these} he brought to Babylon."],
['2Chronicles','2Chr','36','019','19',"And they burnt the house of God, and brake down the wall of Jerusalem, and burnt all the palaces thereof with fire, and destroyed all the goodly vessels thereof."],
['2Chronicles','2Chr','36','020','20',"And them that had escaped from the sword carried he away to Babylon; where they were servants to him and his sons until the reign of the kingdom of Persia:"],
['2Chronicles','2Chr','36','021','21',"To fulfil the word of the LORD by the mouth of Jeremiah, until the land had enjoyed her sabbaths: \emph{for} as long as she lay desolate she kept sabbath, to fulfil threescore and ten years."],
['2Chronicles','2Chr','36','022','22',"Now in the first year of Cyrus king of Persia, that the word of the LORD \emph{spoken} by the mouth of Jeremiah might be accomplished, the LORD stirred up the spirit of Cyrus king of Persia, that he made a proclamation throughout all his kingdom, and \emph{put} \emph{it} also in writing, saying,"],
['2Chronicles','2Chr','36','023','23',"Thus saith Cyrus king of Persia, All the kingdoms of the earth hath the LORD God of heaven given me; and he hath charged me to build him an house in Jerusalem, which \emph{is} in Judah. Who \emph{is} \emph{there} among you of all his people? The LORD his God \emph{be} with him, and let him go up."]
]

def main():
    ProcessVerses('SecondChronicles35',SecondChronicles35)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{Moreover Josiah kept a passover unto the LORD in Jerusalem: and they killed the passover on the fourteenth \emph{day} of the first month.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And he set the priests in their charges, and encouraged them to the service of the house of the LORD,}
[3] \textcolor[cmyk]{0.99998,1,0,0}{And said unto the Levites that taught all Israel, which were holy unto the LORD, Put the holy ark in the house which Solomon the son of David king of Israel did build; \emph{it} \emph{shall} not \emph{be} a burden upon \emph{your} shoulders: serve now the LORD your God, and his people Israel,}
[4] \textcolor[cmyk]{0.99998,1,0,0}{And prepare \emph{yourselves} by the houses of your fathers, after your courses, according to the writing of David king of Israel, and accord

Megiddo & 1 \\ \hline
Moreover & 1 \\ \hline
Moses & 2 \\ \hline
Necho & 2 \\ \hline
Nethaneel & 1 \\ \hline
Nevertheless & 1 \\ \hline
Now & 1 \\ \hline
Put & 1 \\ \hline
Samuel & 1 \\ \hline
Shemaiah & 1 \\ \hline
So & 3 \\ \hline
Solomon & 2 \\ \hline
What & 1 \\ \hline
Zechariah & 1 \\ \hline
\emph{I} & 1 \\ \hline
\emph{after} & 1 \\ \hline
\emph{are} & 2 \\ \hline
\emph{be} & 1 \\ \hline
\emph{blood} & 1 \\ \hline
\emph{busied} & 1 \\ \hline
\emph{cattle} & 2 \\ \hline
\emph{come} & 1 \\ \hline
\emph{day} & 1 \\ \hline
\emph{did} & 1 \\ \hline
\emph{is} & 2 \\ \hline
\emph{it} & 2 \\ \hline
\emph{meddling} & 1 \\ \hline
\emph{offerings} & 1 \\ \hline
\emph{of} & 1 \\ \hline
\emph{one} & 1 \\ \hline
\emph{other} & 1 \\ \hline
\emph{place} & 1 \\ \hline
\emph{shall} & 1 \\ \hline
\emph{small} & 2 \\ \hline
\emph{that} & 1 \\ \hline
\emph{them} & 2 \\ \hline
\emph{they} & 2 \\ \hline
\emph{the} & 1 \\ \hline
\emph{waited} & 1 \\ \hline
\emph{was} & 1 \\ \hline
\emph{were} & 3 \\ \hl